In [43]:
from __future__ import unicode_literals, print_function

import plac
import random
import warnings
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding

In [44]:
LABELS = ['Liability','Notification_type','Termination_period','Force_Majeure_Clause']

#Annotated data
TRAIN_DATA = [
("In this Clause   Event of Force Majeure means an event beyond the control of the A4uthority and the Operator which prevents a Party from complying with any of its obligations under this Contract including but not limited to7.1.1 Neither Party will be liable to the other Party for failure to perform any of its obligations under this Agreement to the extent such performance is hindered delayed or prevented by Force Majeure (except for failure to make payments hereunder)", {"entities":[(228,241,"Liability")]}),
("the Operator which prevents a Party from complying with any of its obligations under this Contract including but not limited to7.1.1 Neither Party will be liable to the other Party for failure to perform any of its obligations under this Agreement to the extent", {"entities":[(133,146,"Liability")]}),
("Party from complying with any of its obligations under this Contract including but not limited to7.1.1 Neither Party will be liable to the other Party for failure to perform any of its obligations under this Agreement to the extent such performance is hindered delayed or prevented by Force Majeure (except for failure to make payments hereunder)", {"entities":[(103,116,"Liability")]}),
("fdsfdfdsfds fdsdsds455454 454 45454 In this Clause   Event of Force Majeure means an event beyond the control of the Authority and the Operator which prevents a Party from fdsfdfds complying with any of its obligations under this Contract including but not limited to7.1.1 Neither Party will be liable to the other Party for failure to perform any of its obligations under this Agreement to the extent such performance is hindered delayed or prevented by Force Majeure (except for failure to make payments hereunder) sdsaddasdasdasasdas", {"entities":[(273,286,"Liability")]}),
("beyond the control of the Authority and the Operator which prevents a Party from complying with any of its obligations under this Contract including but not limited to7.1.1 Neither Party will be liable to the other Party for failure to perform any of its obligations under this Agreement to the extent such performance is hindered delayed or prevented by Force Majeure (except for failure to make payments hereunder) dashjasfhdhjhfdhafkjdhfhdkjahfdakjfhdafkjhdakjfhdahfkjhdakjfhdafkjdhaf 564546af fdafdafda", {"entities":[(173,186,"Liability")]}),
("In this Clause   Event of Force Majeure means an event beyond the control of the Authority and the Operator which prevents a Party from complying with any of its obligations dsfdsfdsfd under this Contract including but not limited to7.1.1 Neither Party will be liable to the other Party for failure to perform any of its obligations under this Agreement sdfdsfdsfds   to the extent such performance is hindered delayed or prevented by Force Majeure (except for failure to make payments hereunder) dsfdsfdsfdsfdsfdsf", {"entities":[(239,252,"Liability")]}),
("lkjlkjlkjlkjlkjlkj In this Clause   Event of Force Majeure means an event beyond the control of the Authority and the Operator which prevents a Party from complying with any of its obligations under this Contract including but not limited to7.1.1 Neither Party will be liable to the other Party for failure to perform any of its obligations under this Agreement to the extent such performance is hindered delayed or prevented by Force Majeure (except for failure to make payments hereunder) kjlkjlkjllllllllllllllllllllllllllllllljlklkjlkkjlkjlkjlkj", {"entities":[(247,260,"Liability")]}),
("Neither Party will be liable to the other Party for failure to perform any of its obligations under this Agreement to the extent such performance is hindered delayed or prevented by Force Majeure (except for failure to make payments hereunder)", {"entities":[(0,13,"Liability")]}),
("Event of Force Majeure means an event beyond the control of the Authority and the Operator which prevents a Party from complying with any of its obligations under this Contract including but not limited to7.1.1 Neither Party will be liable to the other Party for failure to perform any of its obligations under this Agreement to the extent such performance is hindered delayed", {"entities":[(211,224,"Liability")]}),
("A Party which is unable in whole or in part to carry out its obligations under this Agreement due to Force Majeure must provide notice to the other Party. Initial notice may be given orally however written notification with reasonably full particulars of the event or occurrence is required as soon as reasonably possible. 7.1.3 A Party claiming Force Majeure will diligently use all reasonable efforts to remove the cause condition event or circumstance of such Force Majeure will promptly give written notice to the other Party of the termination of such Force Majeure and will resume performance of any suspended obligation as soon as reasonably possible after termination of such Force Majeure.", {"entities":[(198,205,"Notification_type")]}),
("unable in whole or in part to carry out its obligations under this Agreement due to Force Majeure must provide notice to the other Party. Initial notice may be given orally however written notification with reasonably full particulars of the event or occurrence is required as soon as reasonably possible. 7.1.3 A Party claiming Force Majeure will diligently use all reasonable efforts to remove the cause condition event or circumstance of such Force Majeure will promptly", {"entities":[(181,188,"Notification_type")]}),
("dfdfdafdafdafdalk;korturuei[eoqweiwoeiqwi A Party which is unable in whole or in part to carry out its obligations under this Agreement due to Force Majeure must provide notice to the other Party. Initial notice may be given orally however written notification with reasonably full particulars of the event or occurrence is required as soon as reasonably possible. 7.1.3 A Party claiming Force Majeure will diligently use all reasonable efforts to remove the cause condition event or circumstance of such Force Majeure will promptly give written notice to the other Party of the termination of such Force Majeure and will resume performance of any suspended obligation as soon as reasonably possible after termination of such Force Majeure.", {"entities":[(240,247,"Notification_type")]}),
("A Party which is unable in whole or in part to carry out its obligations under this Agreement due to Force Majeure must provide notice to the other Party. Initial notice may be given orally however written notification with reasonably full particulars of the event or occurrence is required as soon as reasonably possible. 7.1.3 A Party claiming Force Majeure will diligently use all reasonable efforts to remove the cause condition event or circumstance of such Force Majeure will promptly give written notice to the other Party of the termination of such Force Majeure and will resume performance of any suspended obligation as soon as reasonably possible after termination of such Force Majeure. sdfdsdsgfagfgfgfgfgewjrlerjqerEPIPOIPOIOPIPOIPOIPOIPOIPOIIPOIPOIWQLKRQELR", {"entities":[(198,205,"Notification_type")]}),
("A GHGHG Party which is unable in whole or in part to carry out its obligations under this Agreement due to Force Majeure must provide notice to the other Party. Initial notice may be given orally however written notification with reasonably full particulars of the event or occurrence is required as soon as reasonably possible", {"entities":[(204,211,"Notification_type")]}),
("FGFGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG LLLLLLLLLLLLLLLLLLLLLLLLLLLLLL MMMMMMMMMMMMMMMMMMMMMMMMMMMM DDDDDDDDDDDDDDDDDDDDDDDD FGDFGDFGDFG      HGDDHDGHD QWEWQPKJK KHJKJHKHJ Party which is unable in whole or in part to carry out its obligations under this Agreement due to Force Majeure must provide notice to the other Party. Initial notice may be given orally however written notification with reasonably full particulars of the event or occurrence is required as soon as reasonably possible. 7.1.3 A Party claiming Force Majeure will diligently use all reasonable efforts to remove the cause condition event or circumstance of such Force Majeure will promptly give written notice to the other Party of the termination of such Force Majeure and will resume performance of any suspended obligation as soon as reasonably possible after termination of such Force Majeure.", {"entities":[(376,383,"Notification_type")]}),
("KLJKLJLKJLKJKLJKLDFJGDFKLGJDFKLGJDFNMNMN,NM NN,N,MN,MN JKLJKLJKL SDSAFDSAFS A Party which is unable in whole or in part to carry out its obligations under this Agreement due to Force Majeure must provide notice to the other Party. Initial notice may be given orally however written notification with reasonably full particulars of the event or occurrence is required as soon as reasonably possible. 7.1.3 A Party claiming Force Majeure will diligently use all reasonable efforts to remove the cause condition event or circumstance of such Force Majeure will promptly give written notice to the other Party of the termination of such Force Majeure GHJGHKGJKJJGGH FGHJGFHGFH LKLKHLKHK  HGJHGJGH", {"entities":[(274,281,"Notification_type")]}),
("Initial notice may be given orally however written notification with reasonably full particulars of the event or occurrence is required as soon as reasonably possible. 7.1.3 A Party claiming Force Majeure will diligently use all reasonable efforts to remove the cause condition event or circumstance of such Force Majeure will promptly give written notice to the other Party of the termination of such Force Majeure and will resume performance of any suspended obligation as soon as reasonably possible after termination of such Force Majeure.", {"entities":[(43,50,"Notification_type")]}),
(" Initial notice may be given orally SGSFGFSGFSGFS however written notification with reasonably full particulars of the event or occurrence is  SDGFSGSDFG ADASDArequired as soon as reasonably possible. 7.1.3 A Party claiming Force Majeure will diligently use all reasonable efforts to remove the cause condition event or circumstance of such Force Majeure will promptly give written notice to the other Party of the termination of such Force Majeure and will resume performance of any suspended obligation as soon as reasonably possible after termination of such Force Majeure.", {"entities":[(58,65,"Notification_type")]}),
("A Party which is unable in whole or in SGFGFSG part to carry out its obligations under this Agreement due to Force Majeure must provide notice to the other Party. Initial notice may be given orally however written notification with reasonably SGFSGFSGFS full particulars of the event or occurrence is required as soon as reasonably possible. 7.1.3 A Party claiming Force Majeure will diligently use all reasonable  SGFGFSG efforts to remove the cause condition event or circumstance of such Force Majeure will promptly give written notice to the other Party of the termination of such Force Majeure and will resume performance of any suspended obligation as soon as reasonably possible after termination of such Force Majeure. DFDSFDF", {"entities":[(206,213,"Notification_type")]}),
("Irrespective of any extension of time if an Event of Force Majeure occurs and its effect continues for a period of 180 days either the Authority or the Operator may give to the other a notice of termination. If Authority is paying fee during Force Majeure then Operator should not have a termination right he is being paid. which shall take effect 28 days after the giving of the notice", {"entities":[(115,118,"Termination_period")]}),
("if an Event of Force Majeure occurs and its effect continues for a period of 180 days either the Authority or the Operator may give to the other a notice of termination. If Authority is paying fee during Force Majeure then Operator should not have a termination right he is being paid. which shall take effect 28 days after the giving of the notice", {"entities":[(77,80,"Termination_period")]}),
("Force Majeure dfdsfdsfd occurs and fdsfdsfds its effect continues for a period of 180 days either the Authority or the Operator may give to the other a notice of termination. If Authority is paying fee during Force Majeure then Operator should not have a termination right he is being paid. which shall take effect 28 days after the giving of the notice", {"entities":[(82,85,"Termination_period")]}),
("extension of time if an Event of Force Majeure occurs and its effect continues for a period of 180 days either the Authority or the Operator may give to the other a notice of termination", {"entities":[(95,98,"Termination_period")]}),
("effect continues for a period of 180 days either the Authority or the Operator may give to the other a notice of termination. If Authority is paying fee during Force Majeure then Operator should not have a termination right he is being paid. which shall take effect 28 days after the giving of the notice", {"entities":[(33,36,"Termination_period")]}),
("Event of Force Majeure occurs and its effect continues for a period of 180 days either the Authority or the Operator may give to the other a notice of termination. If Authority is paying fee during Force Majeure then Operator should not have a termination right he is being paid. which shall take effect 28 days after the giving of the notice", {"entities":[(71,74,"Termination_period")]}),
("dfdsff plplpl iuiruoirueoi iuroewrueoiruewroiuewruewoiruewourro kjfkldsjkldsfjs nfds,mndsf,mdsn . fjdkljdsfkldsjf . df Irrespective of any extension of time if an Event of Force Majeure occurs and its effect continues for a period of 180 days either the Authority or the Operator may give to the other a notice of termination. If Authority is paying fee during Force Majeure then Operator should not have a termination right he is being paid. which shall take effect 28 days after the giving of the notice", {"entities":[(234,237,"Termination_period")]}),
("jfahfjafhj k;kl;ak;;lk pipoipooi. ppoipoipoipoi .idjlfjldjfdklfjdjsfjdl wqewqweqwll of any extension of time if an Event of Force Majeure occurs and its effect continues for a period of 180 days either the Authority or the Operator may give to the other a notice of termination. If Authority is paying fee during Force Majeure then Operator should not have a termination right he is being paid. which shall take effect 28 days after the giving of the notice", {"entities":[(186,189,"Termination_period")]}),
("Irrespective of any extension dfjdklsjklgj pqwiepoweipoqwi of time if an Event of Force Majeure occurs and its effect continues for a period of 180 days either the Authority or the Operator may give to the other a notice of termination. If Authority is paying fee during Force Majeure then Operator should not have a termination right fdsgsdfgfgfsg lplpdlsfej", {"entities":[(144,147,"Termination_period")]}),
("its effect continues for a period of 180 days either the Authority or the Operator may give to the other a notice of termination. If Authority is paying fee during Force Majeure then Operator should not have a termination right he is being paid. which shall take effect 28 days after the giving of the notice", {"entities":[(37,40,"Termination_period")]}),
("Neither the Authority nor the Operator shall be considered in breach of this Contract to the extent that performance of their respective obligations (excluding payment obligations) is prevented by an Event of Force Majeure that arises after the Effective Date.7.2.2 The Party (the   prevented from carrying out its obligations hereunder shall give notice to the other Party of an Event of Force Majeure", {"entities":[(0,38,"Liability")]}),
("riot commotion strikes go slows lock outs or disorder unless solely restricted to employees of the Supplier or of his Subcontractors or 7.1.6 acts or threats of terrorism.7.2 Consequences of Force Majeure Event 7.2.1 Neither the Authority nor the Operator shall be considered in breach of this Contract to the extent that performance of their respective obligations (excluding payment obligations) is prevented by an Event of Force Majeure that arises after the Effective Date", {"entities":[(217,255,"Liability")]}),
("acts or threats of terrorism.7.2 Consequences of Force Majeure Event 7.2.1 Neither the Authority nor the Operator shall be considered in breach of this Contract to the extent that performance of their respective obligations (excluding payment obligations) is prevented by an Event of Force Majeure", {"entities":[(75,113,"Liability")]}),
("lock outs or disorder unless solely restricted to employees of the Supplier or of his Subcontractors or 7.1.6 acts or threats of terrorism.7.2 Consequences of Force Majeure Event 7.2.1 Neither the Authority nor the Operator shall be considered in breach of this Contract to the extent that performance of their respective obligations (excluding payment obligations) is prevented by an Event of Force Majeure that arises after the Effective Date", {"entities":[(185,223,"Liability")]}),
("Force Majeure Event 7.2.1 Neither the Authority nor the Operator shall be considered in breach of this Contract to the extent that performance of their respective obligations (excluding payment obligations) is prevented by an Event of Force Majeure that arises after the Effective Date", {"entities":[(26,64,"Liability")]}),
("Supplier or of his Subcontractors or 7.1.6 acts or threats of terrorism.7.2 Consequences of Force Majeure Event 7.2.1 Neither the Authority nor the Operator shall be considered in breach of this Contract to the extent that performance of their respective obligations (excluding payment obligations) is prevented by an Event of Force Majeure that arises after the Effective Date", {"entities":[(118,156,"Liability")]}),
("Consequences of Force Majeure Event 7.2.1 Neither the Authority nor the Operator shall be considered in breach of this Contract to the extent that performance of their respective obligations (excluding payment obligations) is prevented by an Event of Force Majeure that arises after the Effective Date", {"entities":[(42,80,"Liability")]}),
("dklfldljd poepqwoqwp 312132 weqeqw. wqeweqewq Neither the Authority nor the Operator shall be considered in breach of this Contract to the extent that performance of their respective obligations (excluding payment obligations) is prevented by an Event of Force Majeure that arises after the Effective Date", {"entities":[(46,84,"Liability")]}),
("7.2.1 Neither the Authority nor the Operator shall be considered in breach of this Contract to the extent that performance of their respective obligations (excluding payment obligations) is prevented by an Event of Force Majeure", {"entities":[(6,44,"Liability")]}),
("contamination by radioactivity from any nuclear fuel or from any nuclear waste from the combustion of nuclear fuel radioactive toxic explosive or other hazardous properties of any explosive nuclear assembly or nuclear component of such assembly 7.1.6 acts or threats of terrorism.7.2 Consequences of Force Majeure Event 7.2.1 Neither the Authority nor the Operator shall be considered in breach of this Contract to the extent that performance of their respective obligations (excluding payment obligations) is prevented by an Event of Force Majeure that arises after the Effective Date", {"entities":[(326,364,"Liability")]}),
("The Contract Period shall be extended by a period of time equal to the period of interruption caused by an Event of Force Majeure.1.2.6 OR 1.2.7 7.2.8 Within 30 business days of occurrence of a Force Majeure Event the affected Party will give written notice thereof to the other Party giving sufficient details of the cause and to the extent that it is likely to affect its performance.The affected Party will be entitled to an extension of time equal to the duration of the Force Majeure Event", {"entities":[(243,250,"Notification_type")]}),
("caused by an Event of Force Majeure.1.2.6 OR 1.2.7 7.2.8 Within 30 business days of occurrence of a Force Majeure Event the affected Party will give written notice thereof to the other Party giving sufficient details of the cause and to the extent that it is likely to affect its performance.The affected Party will be entitled to an extension of time equal to the duration of the Force Majeure Event", {"entities":[(149,156,"Notification_type")]}),
("Within 30 business days of occurrence of a Force Majeure Event the affected Party will give written notice thereof to the other Party giving sufficient details of the cause and to the extent that it is likely to affect its performance", {"entities":[(92,99,"Notification_type")]}),
("the affected Party will give written notice thereof to the other Party giving sufficient details of the cause and to the extent that it is likely to affect its performance.The affected Party will be entitled to an extension of time equal to the duration of the Force Majeure Event", {"entities":[(29,36,"Notification_type")]}),
("Event of Force Majeure.1.2.6 OR 1.2.7 7.2.8 Within 30 business days of occurrence of a Force Majeure Event the affected Party will give written notice thereof to the other Party giving sufficient details sgsgsfgfsgfs", {"entities":[(136,143,"Notification_type")]}),
("afjlafjdafj piwpoeiwp ghgh. lghlghklh.  ei The Contract Period shall be extended by a period of time equal to the period of interruption caused by an Event of Force Majeure.1.2.6 OR 1.2.7 7.2.8 Within 30 business days of occurrence of a Force Majeure Event the affected Party will give written notice thereof to the other Party giving sufficient details of the cause and to the extent that it is likely to affect its performance.The affected Party will be entitled to an extension of time equal to the duration of the Force Majeure Event", {"entities":[(286,293,"Notification_type")]}),
("The affected Party will be entitled to an extension of time equal to the duration of the Force Majeure Event for the performance of its obligations.7.3 Optional Termination Payment and Release Irrespective of any extension of time if an Event of Force Majeure occurs and its effect continues for a period of 90 days either the Authority or the Operator may give to the other a notice of termination.If Authority is paying fee during Force Majeure then Operator should not have a termination right", {"entities":[(308,310,"Termination_period")]}),
("Force Majeure Event for the performance of its obligations.7.3 Optional Termination Payment and Release Irrespective of any extension of time if an Event of Force Majeure occurs and its effect continues for a period of 90 days either the Authority or the Operator may give to the other a notice of termination.If Authority is paying fee during Force Majeure then Operator should not have a termination right", {"entities":[(219,221,"Termination_period")]}),
("7.3 Optional Termination Payment and Release Irrespective of any extension of time if an Event of Force Majeure occurs and its effect continues for a period of 90 days either the Authority or the Operator may give to the other a notice of termination.If Authority is paying fee during Force Majeure then Operator should not have a termination right", {"entities":[(160,162,"Termination_period")]}),
("Irrespective of any extension of time if an Event of Force Majeure occurs and its effect continues for a period of 90 days either the Authority or the Operator may give to the other a notice of termination", {"entities":[(115,117,"Termination_period")]}),
("the duration of the Force Majeure Event for the performance of its obligations.7.3 Optional Termination Payment and Release Irrespective of any extension of time if an Event of Force Majeure occurs and its effect continues for a period of 90 days either the Authority or the Operator may give to the other a notice of termination.If Authority is paying", {"entities":[(239,241,"Termination_period")]}),
(" Event for the performance of its obligations.7.3 Optional Termination Payment and Release Irrespective of any extension of time if an Event of Force Majeure occurs and its effect continues for a period of 90 days either the Authority or the Operator may give to the other a notice of termination.If Authority is paying fee during Force Majeure then Operator should not have a termination right", {"entities":[(206,208,"Termination_period")]}),
("The affected Party will kjfdkaflj ljfdalfjldfjkda lfkdjfkjdklfjkl be entitled to an extension of time equal to the duration of the Force afadfdafaf Majeure Event for the performance of its obligations.7.3 Optional Termination Payment and Release Irrespective of any extension of time if an Event of Force Majeure occurs and its effect continues for a period of 90 days either the Authority or the Operator may give to the other a notice of termination.If Authority is paying fee during Force Majeure then Operator should not have a termination right", {"entities":[(361,363,"Termination_period")]}),
("Termination Payment and Release Irrespective of any extension of time if an Event of Force Majeure occurs and its effect continues for a period of 90 days either the Authority or the Operator may give to the other a notice of termination dfjdkafjldlj qpoiwoiwoe pewpoeipqw qenf,mnd,mnf 3212", {"entities":[(147,149,"Termination_period")]}),
("The affected Party will be entitled to an extension of tdfdfkd w;kq;eqk  qppwoepwe lll ,,, . dfdfdfdfdime equal to the duration of the Force Majeure Event for the performance of its obligations.7.3 Optional Termination Payment and Release Irrespective of any extension of time if an Event of Force Majeure occurs and its effect continues for a period of 90 days either the Authority or the Operator may give to the other a notice of termination.If Authority is paying fee during Force Majeure then Operator should not have a termination right", {"entities":[(354,356,"Termination_period")]}),
(";lkada popowrii kdlkfldfk 313213 Irrespective of any extension of time if an Event of Force Majeure occurs and its effect continues for a period of 90 days either the Authority or the Operator may give to the other a notice of termination.If Authority is paying fee during Fordfdgdfsgfgfsgfg gfgfgfsce Majeure then Operator should not have a termination right", {"entities":[(148,150,"Termination_period")]}),
("if an Event of Force Majeure occurs and its effect continues for a period of 90 days either the Authority or the Operator may give to the other a notice of termination.If Authority is paying fee during Force Majeure then Operator should not have a termination right", {"entities":[(77,79,"Termination_period")]}),
("nonclaiming Party may terminate this Agreement.(5)  requests demands and other communications under this Agreement shall be in writing and shall be deemed to have been duly given if (a) delivered by hand and receipted for by the party to whom said notice or other communication shall have been directed (b) mailed by certified or registered mail with postage prepaid on the third business day after the date on which it is so mailed", {"entities":[(127,134,"Notification_type")]}),
("requests demands and other communications under this Agreement shall be in writing and shall be deemed to have been duly given if (a) delivered by hand and receipted for by the party to whom said notice or other communication shall have been directed (b) mailed by certified or registered mail with postage prepaid on the third business day after the date on which it is so mailed", {"entities":[(75,82,"Notification_type")]}),
("djfldf perieporri kldfjkldf dnf,mdnfd,ms ;dklf;dkf;ld f ljkljjjjj nonclaiming Party may terminate this Agreement.(5)  requests demands and other communications under this Agreement shall be in writing and shall be deemed to have been duly given if communication shall have been directed (b) mailed by certified or registered mail with postage prepaid on the third business day after the date on which it is so mailed", {"entities":[(193,200,"Notification_type")]}),
("demands and other communications under this Agreement shall be in writing and shall be deemed to have been duly given if (a) delivered by hand and receipted for by the party to whom said notice or other communication shall have been directed", {"entities":[(66,73,"Notification_type")]}),
("lfjdkfj peqiropri mnrmenr ,,mrn,er 123 erqrereqrqe ghnbrehklndsmnv;  requests demands and other communications under this Agreement shall be in writing and shall be deemed to have been duly given if (a) delivered by hand and receipted for by the party to whom said notice or other communication shall have been directed (b) mailed by certified or registered mail with postage prepaid on the third business day after the date on which it is so mailed", {"entities":[(144,151,"Notification_type")]}),
("relrerkj mnv,mfngfngmgn,fs communications under this Agreement shall be in writing and shall be deemed to have been duly given if (a) delivered by hand and receipted for by the party fadfdf lkelrkeopep", {"entities":[(75,82,"Notification_type")]}),
("nonclaiming Party may terminate  jkfjdalfkjdaf dlafjlfkjd mnfdma,fdna reorip this Agreement.(5)  requests demands and other communications under this Agreement shall be in writing and shall be deemed to have been duly given if (a) delivered by hand and receipted for by the party to whom said notice or other communication shall have been directed (b) mailed by certified or registered mail with postage prepaid on the third business day after the date on which it is so mailed", {"entities":[(172,179,"Notification_type")]}),
("dfdfdf mgnf,mngfgbnfmngbdf ljgfkljireoiruerydnjvbpt498jmgb  demands and other communications under this Agreement shall be in writing and shall be deemed to have been duly given if (a) delivered by hand and receipted for by the party to whom said notice or other communication shall have been directed (b) mailed by certified or registered mail with postage prepaid on the third business day after the date on which it is so mailed", {"entities":[(126,133,"Notification_type")]}),
("(5)  requests demands and other communications under this Agreement shall be in writing and shall be deemed to have been duly given if (a) delivered by hand and receipted for by the party to whom said notice or other communication shall have been directed (b) mailed by", {"entities":[(80,87,"Notification_type")]}),
("fdasfd mgnmgn 123 eewtlkjwtljoigfgnfsgnfsklgfskltgjkjtg Agreement.(5)  requests demands and other communications under this Agreement shall be in writing and shall be deemed to have been duly given if (a) delivered by hand and receipted for by the party to whom said notice or other communication shall have been directed (b) mailed by  sdhgsjghkjsg trwtuirpp msgnmsg sd certified or registered mail with postage prepaid on the third business day after the date on which it is so mailed", {"entities":[(146,153,"Notification_type")]}),
("nonclaiming Party may terminate this Agreement.(5) fdhgj gklgklskgjsk mng,mfsgnmg requests demands and other communications under this Agreement shall be in writing and shall be deemed to have been duly given if (a) delivered by hand and receipted for by the party to whom said notice or other communication shall have been directed (b) mailed by certified or registered mail with postage prepaid on the third business day after the date on which it is so mailed", {"entities":[(157,164,"Notification_type")]}),
("Parties shall consult together in relation to the above matters following the occurrence of a Force Majeure Event.(4) Force Majeure Event affects the performance of the claiming Party for 180 consecutive days the nonclaiming Party may terminate this Agreement.(5)  requests demands and other communications under this Agreement shall be in writing", {"entities":[(188,191,"Termination_period")]}),
("Force Majeure Event affects the performance of the claiming Party for 180 consecutive days the nonclaiming Party may terminate this Agreement.(5)  requests demands and other communications under this Agreement", {"entities":[(70,73,"Termination_period")]}),
("following the occurrence of a Force Majeure Event.(4) Force Majeure Event affects the performance of the claiming Party for 180 consecutive days the nonclaiming Party may terminate this Agreement", {"entities":[(124,127,"Termination_period")]}),
("fladf kljlfkd fjljfda fjdk Force Majeure Event affects the performance of the claiming Party for 180 consecutive days the nonclaiming Party may terminate this Agreement.(5)  requests demands and other communications under this Agreement shall be in writing", {"entities":[(97,100,"Termination_period")]}),
("Parties shall dafjldkfja nmdnf,daf dfkjdajkfla dlkfjdaklfjda fd,manfd,manfd,mfnda,mfdna  fkldajflda   dakfjdklfj consult together in relation to the above matters following the occurrence of a Force Majeure Event.(4) Force Majeure Event affects the performance of the claiming Party for 180 consecutive days the nonclaiming Party may terminate this Agreement.(5)  requests demands and other communications under this Agreement shall be in writing", {"entities":[(287,290,"Termination_period")]}),
("fsgfsgsf sfgfsg nmnmnxbvcmnbvljdkf pio xvm following the occurrence of a Force Majeure Event.(4) Force Majeure Event affects the performance of the claiming Party for 180 consecutive days the nonclaiming Party may terminate this Agreement dgfdggs klgjsklgj sg fsgfsgs", {"entities":[(167,170,"Termination_period")]}),
("dserrtret bnmnb poipoi Force Majeure Event affects the performance of the claiming Party for 180 consecutive days the nonclaiming Party may terminate this Agreement.(5)  requests demands and other communications under this Agreement shall be in writing", {"entities":[(93,96,"Termination_period")]}),
("Agreement and the Ancillary Agreements as applicable as soon as reasonably practicable.7.2 Consequences of Force Majeure Event7.2.1 Neither the Authority nor the Operator shall be considered in breach of this Contract to the extent that performance of their respective obligations (excluding payment obligations) is prevented by an Event of Force Majeure that arises after the Effective Date", {"entities":[(132,170,"Liability")]}),
("Agreement and the Ancillary Agreements dfjldfj poripo mc,bvnmnbxcn mbvxm rerouero as applicable as soon as reasonably practicable.7.2 Consequences of Force Majeure Event7.2.1 Neither the Authority nor the Operator shall be considered in breach of this Contract to the extent that performance of their respective obligations", {"entities":[(175,213,"Liability")]}),
("Agreement  afdafdaf dfdafand the Ancillary Agreements as applicable as afadfdafdafdaf soon as reasonably practicable.7.2 Consequences of Force Majeure Event7.2.1 Neither the Authority nor the Operator shall be considered in breach of this Contract to the extent that performance of their respective obligations (excluding payment obligations) is prevented by an Event of Force Majeure that arises after the Effective Date", {"entities":[(162,200,"Liability")]}),
("Consequences of Force Majeure Event7.2.1 Neither the Authority nor the Operator shall be considered in breach of this Contract to the extent that performance of their respective obligations (excluding payment obligations) is prevented by an Event of Force Majeure that arises after the Effective Date", {"entities":[(41,79,"Liability")]}),
("7.2.1 Neither the Authority nor the Operator shall be considered in breach of this Contract to the extent that performance of their respective obligations (excluding payment obligations) is prevented by an adfdaf dafda fdafdafda  Event of Force Majeure that arises after the Effective Date", {"entities":[(6,44,"Liability")]}),
("A Party claiming the benefit of this provision shall as soon as reasonably practicable after the occurrence of any such event (a) provide written notice to the other Party of the nature and extent of any such Force Majeure condition and (b) use commercially reasonable efforts to remove any such causes and resume performance under this Agreement", {"entities":[(138,145,"Notification_type")]}),
("the occurrence of any such event (a) provide written notice to the other Party of the nature and extent of any such Force Majeure condition and (b) use commercially reasonable efforts to remove any such causes and resume performance under this Agreement", {"entities":[(45,52,"Notification_type")]}),
("dfkja menr,m ;k;lk ieiporipori nbvcbvczbv wehwjkhw rerje Party claiming the benefit  of this provision shall as soon as reasonably practicable after the occurrence of any such event (a) provide written notice to the other Party of the nature and extent of any such Force Majeure condition and (b) use commercially reasonable efforts to remove any such causes and resume performance under this Agreement", {"entities":[(194,201,"Notification_type")]}),
("provide written notice to the other Party of the nature and extent of any such Force Majeure condition ttg v hdgfh h yhryytyxxcftreykuipmnmbv ", {"entities":[(8,15,"Notification_type")]}),
("freq qr n,mvbnvmb piporw mbn,mfng jhtkl A Party claiming the benefit of this provision shall as soon as reasonably practicable etettrwtrwt after the occurrence of any such event (a) provide written notice to the other Party of the nature and extent of any such Force Majeure condition and (b) use commercially reasonable efforts to remove any such causes and resume performance under this Agreement", {"entities":[(190,197,"Notification_type")]}),
("A Party claiming the  hdgfhdgf bmvmvm rerwtwret benefit of this provision shall as soon as reasonably practicable after the occurrence of any such event (a) provide written notice to the other Party of the nature and extent of any such Force Majeure condition and (b)  gfgr rtyrtrw vbvb oipiy rqewreq bmbnm use commercially reasonable efforts to remove any such causes and resume performance under this Agreement", {"entities":[(165,172,"Notification_type")]}),
(" gfsgfs vnn pup Party claiming the benefit of this provision shall as soon as ghg gmm  pipo rn bdmn  reasonably practicable after the occurrence of any such event (a) provide written notice to the other Party of the nature and extent of any such Force Majeure condition and (b) use commercially reasonable efforts to remove any such causes and resume performance under this Agreement sgfsgfs rtrtrew m,bn,nb,turty yrturtuzx sds ", {"entities":[(175,182,"Notification_type")]}),
("Termination Payment and ReleaseA Party may terminate this Agreement upon providing written notice to the other Party if the other Party is affected by a Force Majeure event which cannot be removed overcome or abated within 3 continuous months or 90 days (or within such other period as the Parties jointly shall agree in writing) from the initial date of such Force Majeure event", {"entities":[(246,248,"Termination_period")]}),
("Termination Payment and ReleaseA Party may terminate this Agreement upon providing written notice to the other Party if the other Party is affected by a Force Majeure event which cannot be removed overcome or abated within 3 continuous months or 90 days (or within such other period as the Parties jointly shall agree in writing) from the initial date of such Force Majeure event", {"entities":[(246,248,"Termination_period")]}),
("Party may terminate this Agreement upon providing written notice to the other Party if the other Party is affected by a Force Majeure event which cannot be removed overcome or abated within 3 continuous months or 90 days (or within such other period as the Parties jointly shall agree in writing) from the initial date of such Force Majeure event", {"entities":[(213,215,"Termination_period")]}),
("dbmdnbf ureoru mnbzfmn qeipwoi cmnbmn erhekjrh bmdbf iereoriuoi r ddsg Party may terminate this Agreement upon providing written notice to the other Party if the other Party is affected by a Force Majeure event which cannot be removed overcome or abated within 3 continuous months or 90 days (or within such other period as the Parties jointly shall agree in writing)  sdgsgsfg fgfgfsgfsgfsgfsgfs", {"entities":[(284,286,"Termination_period")]}),
("Termination  dfdsgsg mbemn ttiporwtiporw nbetmn pitwpoirrw t bmrmnw trmnwbtn Payment and ReleaseA Party may terminate this Agreement trwtrwtrw cvbxxvcbxvb upon providing written notice to the other Party if the other Party is affected by a Force Majeure event which cannot be removed overcome or abated within 3 continuous months or 90 days (or within such other period as the Parties jointly shall agree in writing) from the initial date of such Force Majeure event", {"entities":[(333,335,"Termination_period")]}),
("Party if the other Party is affected by a Force Majeure event which cannot be removed overcome or abated within 3 continuous months or 90 days (or within such other period as the Parties jointly shall agree in writing) from the initial date of such Force Majeure event  dgfsgfsgfs sfgrgrwtr4wtqwtkjuppgfhnghn dghdghdghrtuiyrui ", {"entities":[(135,137,"Termination_period")]}),
("Termination Payment and ReleaseA Party may rtrwtrw vbxvbxv sfgsfgsf terminate this Agreement upon providing written notice to the other Party if the other Party is affected by a Force Majeure event which cannot be removed overcome or abated within 3 continuous months or 90 days rwtrwtrwrw rtrwtrwt piwp tmnrbqerb qerq", {"entities":[(271,273,"Termination_period")]}),
("terminate this Agreement upon providing written notice to the other Party if the other Party is affected by a Force Majeure event which cannot be removed overcome or abated within 3 continuous months or 90 days (or within such other period as the Parties jointly shall agree in writing) from the initial date of such Force Majeure event", {"entities":[(203,205,"Termination_period")]}),
("Termination Payment tgrwtrwt rtrwtrwtrw vbxvb ereqrqer bnbnb and ReleaseA Party may terminate this Agreement upon providing written notice to the other Party if the other Party is affected by a Force Majeure event which cannot be removed overcome or abated within 3 continuous months or 90 days (or within such other period as the Parties jointly shall agree in writing) from the initial date of such Force Majeure event", {"entities":[(287,289,"Termination_period")]}),
("rerq vcbncv Party is affected by a Force Majeure event which cannot be removed overcome or abated within 3 continuous months or 90 days (or within such other period as the Parties jointly shall agree in writing) from the initial date of such Force Majeure event dgsfgfsgfsgfs rwtrtw ", {"entities":[(128,130,"Termination_period")]}),
("Within 24 hours of the occurrence of a Force Majeure Event the affected party shall notify the other party of the occurrence by sending either (i) an email message or(ii) oral notice via telephone (iii) a fax message to the other party.7.1.8 The time for performance required of the affected party shall be extended by the period of such delay", {"entities":[(150,216,"Notification_type")]}),
("affected party shall notify the other party of the occurrence by sending either (i) an email message or(ii) oral notice via telephone (iii) a fax message to the other party.7.1.8 The time for performance required of the affected party shall be extended", {"entities":[(87,153,"Notification_type")]}),
("Force Majeure Event the affected party shall notify the other party of the occurrence by sending either (i) an email message or(ii) oral notice via telephone (iii) a fax message to the other party", {"entities":[(111,177,"Notification_type")]}),
("notify the other party of the occurrence by sending either (i) an email message or(ii) oral notice via telephone (iii) a fax message to the other party.7.1.8 The time for performance required of the affected party", {"entities":[(66,132,"Notification_type")]}),
("occurrence of a Force Majeure Event the affected party shall notify the other party of the occurrence by sending either (i) an email message or(ii) oral notice via telephone (iii) a fax message to the other party.7.1.8 The time for performance required of the affected party shall be extended by the period of such delay", {"entities":[(127,193,"Notification_type")]}),
("fdglj djglk gjpetuipo imvnxcmb,nxc,m ertuptuir  poioi ehnr,qewbnm,zbn zoccurrence of a Force Majeure Event the affected party shall notify the other party of the occurrence by sending either (i) an email message or(ii) oral notice via telephone (iii) a fax message to the other party.7.1.8 The time for performance required of the affected party shall be extended by the period of such delay", {"entities":[(198,264,"Notification_type")]}),
("other party of the occurrence by sending either (i) an email message or(ii) oral notice via telephone (iii) a fax message to the other party.7.1.8 The time for performance required of the affected party shall be extended by the period of such delay", {"entities":[(55,121,"Notification_type")]}),
(",nsm gereopri cnbv,mcn goirepori dfjdakflj opiretgjfskjgh erbn etpoi dfbnerejpq  the occurrence of a Force Majeure Event the affected party shall notify the other party of the occurrence by sending either (i) an email message or(ii) oral notice via telephone (iii) a fax message to the other party.7.1.8 The time for performance required of the affected party shall be extended by the period of such delay", {"entities":[(212,278,"Notification_type")]}),
("Within erttrw fvbvbvxb hours of the occurrence of a Force Majeure Event the affected party shall notify the other party of the occurrence by sending either (i) an email message or(ii) oral notice via telephone (iii) a fax message to the other party.7.1.8 deftrewt dc gfsgrwtrwt areerqe jhkulyi fvdghdgh The time for performance required of the affected party shall be extended by the period of such delay", {"entities":[(163,229,"Notification_type")]}),
("tgrwt mnbrmt tpriw pfbngmbfs gfjsgh mntbrmntb Force Majeure Event the affected party shall notify the other party of the occurrence by sending either (i) an email message or(ii) oral notice via telephone (iii) a fax message to the other party.7.1.8 The time for performance required of the affected party  rtrtw vb nxnvnhgtyhtey ytey shall be extended by the period of such delay", {"entities":[(157,223,"Notification_type")]}),
("by sending either (i) an email message or(ii) oral notice via telephone (iii) a fax message to the other party.7.1.8 The time for performance required of the affected party shall be extended dethjet etip tmntrwbt tpoito ptnbt ", {"entities":[(25,91,"Notification_type")]}),
("The Parties shall consult together in relation to the above matters following the occurrence of a Force Majeure Event.(4) Force Majeure Event affects the performance of the claiming Party for ninety (90) consecutive days the nonclaiming Party may terminate this Agreement or an affected Purchase Order.Schedule 1 Services Deliverables and Pricing Description ", {"entities":[(200,202,"Termination_period")]}),
("consult together in relation to the above matters following the occurrence of a Force Majeure Event.(4) Force Majeure Event affects the performance of the claiming Party for ninety (90) consecutive days the nonclaiming Party may terminate this Agreement or an affected Purchase Order.Schedule 1 Services Description ", {"entities":[(182,184,"Termination_period")]}),
("following the occurrence of a Force Majeure Event.(4) Force Majeure Event affects the performance of the claiming Party for ninety (90) consecutive days the nonclaiming Party may terminate this Agreement or an affected Purchase Order", {"entities":[(132,134,"Termination_period")]}),
("Force Majeure Event.(4) Force Majeure Event affects the performance of the claiming Party for ninety (90) consecutive days the nonclaiming Party may terminate this Agreement or an affected Purchase Order.Schedule 1 Services Deliverables and Pricing Description ", {"entities":[(102,104,"Termination_period")]}),
("fadfe ebmnbr epipw  ndmbg hjrkrj  in relation to the above matters following the occurrence of a Force Majeure Event.(4) Force Majeure Event affects the performance of the claiming Party for (90) consecutive days the nonclaiming Party may terminate this Agreement  fgfgfsg rtrwtrw ", {"entities":[(192,194,"Termination_period")]}),
("The dff et bmtb pi poer nzmb wqjewl keiueiru dnbfdn qerpri dbndae,m ,e Parties shall consult together in relation to the above matters following the occurrence of a Force Majeure Event.(4) Force Majeure Event affects the performance of the claiming Party for ninety (90) consecutive days the nonclaiming Party may terminate this Agreement or an affected Purchase Order.Schedule 1 Services Deliverables  etretjrewt r,mtnrw,t rwijurwtiu ", {"entities":[(267,269,"Termination_period")]}),
("rere vb rew v rt consult together in relation to the above matters following the occurrence of a Force Majeure Event.(4) Force Majeure Event affects the performance of the claiming Party for ninety (90) consecutive days the nonclaiming Party may terminate this Agreement or an affected  etrw tvf ryrwy rwyrwyrwy bmnhfjjsrqw upu Purchase Order.Schedule 1 Services Deliverables and Pricing Description ", {"entities":[(199,201,"Termination_period")]}),
("The Parties shall consul fsgg fsgfs gfsg fsgt together in relation to the above matters  rrwtyr fb gfbf following the occurrence of a Force Majeure Event.(4) Force Majeure Event affects the performance of the claiming Party for ninety (90) consecutive days dg rwtr hfh  the nonclaiming Party may terminate this Agreement or an affected Purchase Order.Schedule 1 Services Deliverables and Pricing Description ", {"entities":[(236,238,"Termination_period")]}),
("Event affects the performance of the claiming Party for ninety (90) consecutive days the nonclaiming Party may terminate this Agreement or an affected Purchase Order.Schedule 1 Services Deliverables and Pricing Description ", {"entities":[(64,66,"Termination_period")]}),
("fearae;l m, nm potiportir dma,.mwreope,mn etjrei mtne,tForce Majeure Event affects the performance of the claiming Party for ninety (90) consecutive days the nonclaiming Party may terminate this Agreement or an affected Purchase Order.Schedule 1 Services Deliverables and Pricing Description ", {"entities":[(133,135,"Termination_period")]}),
("rle mne,t lijuit occurrence of a Force Majeure Event.(4) Force Majeure Event affects the performance of the claiming Party for ninety (90) consecutive days the nonclaiming Party may terminate this Agreement or an affected Purchase Order.Schedule 1  trm,tnrt tjrwlt t rwtrw", {"entities":[(135,137,"Termination_period")]}),
("The eqrqe bvbf rtrwrew  Parties  er shall consult together in relation to the above matters following the occurrence of a Force Majeure Event.(4) Force Majeure Event affects the performance of the claiming Party for ninety (90) consecutive days the nonclaiming Party may terminate this Agreement  etrwt vn ndg or an affected Purchase Order.Schedule 1 Services Deliverables and Pricing Description ", {"entities":[(224,226,"Termination_period")]}),
("causes beyond the reasonable control of the affected party (hereinafter referred to as aEvent) the party who has been so affected shall immediately give written notice to the other party of the occurrence of the Force Majeure Event (with a description in reasonable detail of the circumstances causing such Event) and shall do everything reasonably possible to resume performance", {"entities":[(153,160,"Notification_type")]}),
("reasonable control of the affected party (hereinafter referred to as aEvent) the party who has been so affected shall immediately give written notice to the other party of the occurrence of the Force Majeure Event (with a description in reasonable detail of the circumstances causing such Event) ", {"entities":[(135,142,"Notification_type")]}),
("the party  dfdf ertt fcv bgfs gwho has been so affected shall immediately give written notice to the other party of the occurrence of the Force Majeure Event (with a description in reasonable detail of the circumstances causing such Event) and shall do everything reasonably possible dg rwtr trwtrwt  to resume performance", {"entities":[(79,86,"Notification_type")]}),
("frareq ereqrqe fhgf hcontrol of the affected party rer eett trw(hereinafter referred to as aEvent) the party who has been so affected shall immediately give written notice to the other party of the occurrence of the Force Majeure Event (with a description in reasonable detail of the circumstances causing such Event) and shall do everything reasonably possible to resume performance", {"entities":[(157,164,"Notification_type")]}),
("rerqe fbb ytyte yteh dhdthe party who has been so affected shall immediately give written notice to the other party of the occurrence of the Force Majeure Event (with a description in reasonable detail of the circumstances causing such Event) ttwtwr ggvndn tyteyte re ", {"entities":[(82,89,"Notification_type")]}),
("causes beyond ryrwrwyrw wry the reasonable control of  erttrw vngndg yetythe affected party (hereinafter referred to as aEvent) the party who has been so affected shall immediately give written notice to the other party of the occurrence of the Force Majeure Event (with a description in reasonable detail of the circumstances causing such Event) and shall do everything reasonably possible to resume performance ettt bnbgfndghteyhte ftrwetqeqw teryopyd hdghe", {"entities":[(186,193,"Notification_type")]}),
("affected party continues to use commercially reasonable efforts to recommence performance whenever and to whatever extent possible without delay.If the period of nonperformance exceeds thirty (30) days from the receipt of written notice of the Force Majeure Event the party whose ability to perform has not been so affected may by giving written notice terminate this Contract", {"entities":[(193,195,"Termination_period")]}),
("affected party continues to use commercially reasonable efforts to recommence performance whenever and to whatever extent possible without delay.If the period of nonperformance exceeds thirty (30) days from the receipt of written notice of the Force Majeure Event the party whose ability to perform has not been so affected may by giving written notice terminate this Contract", {"entities":[(193,195,"Termination_period")]}),
("to recommence performance whenever and to whatever extent possible without delay.If the period of nonperformance exceeds thirty (30) days from the receipt of written notice of the Force Majeure Event the party whose ability to perform has not been so affected may by giving written notice terminate this Contract", {"entities":[(129,131,"Termination_period")]}),
("If the period of nonperformance exceeds thirty (30) days from the receipt of written notice of the Force Majeure Event the party whose ability to perform has not been so affected may by giving written notice terminate", {"entities":[(48,50,"Termination_period")]}),
("eruir vd,msvbn,ms pwrie prnsbdkjahfdjgqwrwoei dmnbdsmnbetpei poa erber re period of nonperformance exceeds thirty (30) days from the receipt of written notice of the Force Majeure Event the party whose ability to perform has not been so affected may by giving written notice terminate this Contract ere fbf bfsgrtw ", {"entities":[(115,117,"Termination_period")]}),
("erereq bcsb rtet ttqe performance whenever and to whatever extent possible without delay.If the period of nonperformance exceeds thirty (30) days from the receipt of written notice of the Force Majeure Event the party whose ability to perform has not been so affected may by giving written notice terminate this Contract", {"entities":[(137,139,"Termination_period")]}),
("erklj em,n proiqwp If the period of nonperformance exceeds thirty (30) days from the receipt of written notice of the Force Majeure Event the party whose ability to perform has not been so affected may by giving  ereqrq cfbg fgeeqt qetewritten notice terminate this Contract", {"entities":[(67,69,"Termination_period")]}),
("affected qerqerqe qerqe nhf tupo8iu rw party ereqr continues to use commercially reasonable efforts to recommence performance whenever and to whatever extent possible without delay.If the period of nonperformance exceeds thirty (30) days from the receipt of written notice of the Force Majeure Event the party whose ability to perform has not been so affected may by giving written notice terminate this Contract", {"entities":[(229,231,"Termination_period")]}),
("affected party continues to qrqerqe qe use commercially reasonable efforts to recommence performance  qerqerqe daswhenever and to whatever extent possible without delay.If the period of nonperformance exceeds thirty (30) days from the receipt of written notice of the Force Majeure Event  ererqe gfsg tgetqe t aaethe party whose ability to perform has not been so affected may by giving written notice terminate this Contract", {"entities":[(217,219,"Termination_period")]}),
(" whatever extent possible without delay.If the period of nonperformance exceeds thirty (30) days from the receipt of written notice of the Force Majeure Event the party whose ability to perform has not been so affected may by giving  eer written notice terminate this Contract eqr n myipupu ttrwt sfqeqgf jhfj", {"entities":[(88,90,"Termination_period")]}),
(" erew gbfs ghrfstrw  use commercially reasonable efforts to recommence performance whenever and to whatever extent possible without delay.If the period of nonperformance exceeds thirty (30) days from the receipt of written notice of the Force Majeure Event rwtrw rtwtrw  the party rtr trw", {"entities":[(186,188,"Termination_period")]}),
("earthquake accident acts of the public enemy war terrorism rebellion insurrection riot and invasion.In the event of the Party affected shall promptly notify the other Party either by telephoneemailor any oral confirmation and shall use reasonable commercial efforts to eliminate cure or overcome such event and to resume performance of its obligation", {"entities":[(183,221,"Notification_type")]}),
("enemy war terrorism rebellion insurrection riot and invasion.In the event of the Party affected shall promptly notify the other Party either by telephoneemailor any oral confirmation and shall use reasonable commercial efforts to eliminate cure or overcome such event and to resume performance of its obligation", {"entities":[(144,182,"Notification_type")]}),
("earthquake  erer popou cbv ncaccident acts of the public enemy war terrorism rebellion insurrection riot and invasion.In the event of the Party affected shall promptly notify the other Party either by telephoneemailor any oral confirmation and shall use reasonable commercial  sgfsgsf sgsgfsg efforts to eliminate cure or overcome such event and to resume performance of its obligation", {"entities":[(201,239,"Notification_type")]}),
("public enemy war terrorism rebellion insurrection riot and invasion.In the event of force majeure event the Party affected shall promptly notify the other Party either by telephoneemailor any oral confirmation and shall use reasonable commercial efforts to eliminate cure or overcome such event and to resume performance of its obligation", {"entities":[(171,209,"Notification_type")]}),
("In the event of the Party affected shall promptly notify the other Party either by telephoneemailor any oral confirmation and shall use reasonable commercial efforts to eliminate cure or overcome such event", {"entities":[(83,121,"Notification_type")]}),
("fdafe r[poeroqepo nrbenmrb enr,me rn,em12 1 e ereqr acts of the public enemy war terrorism rebellion insurrection riot and invasion.In the event of the Party affected shall promptly notify the other Party either by telephoneemailor any oral confirmation and shall use reasonable commercial efforts to eliminate cure or overcome such event and to resume performance of its obligation", {"entities":[(215,253,"Notification_type")]}),
("accident acts of the public enemy war terrorism rebellion insurrection riot and invasion.In the event of the Party affected shall promptly notify the other Party either by telephoneemailor any oral confirmation and shall use reasonable", {"entities":[(172,210,"Notification_type")]}),
("ew w dajhfajhf fdamnbdamnf 1213af earthquake accident acts of the public enemy war terrorism afdda45da4f56 af2132  rebellion insurrection riot and invasion.In the event of the Party affected shall promptly notify the other Party either by telephoneemailor any oral confirmation and shall use reasonable commercial efforts to eliminate cure or overcome such event and to resume performance of its obligation", {"entities":[(239,277,"Notification_type")]}),
("war terrorism rebellion insurrection riot and invasion aere bv dgh hdfh d.In the event of the Party affected shall promptly notify the other Party either by telephoneemailor any oral confirmation and shall use reasonable commercial efforts to dhfdgf hd dgdgh  eliminate cure or overcome such event and to resume performance of its obligation", {"entities":[(157,195,"Notification_type")]}),
("ere fgfgs nbcgfndg ewqeqw fsgfsg ghgdh event of the Party affected shall promptly notify the other Party either by telephoneemailor any oral confirmation and shall use reasonable commercial efforts to eliminate cure or overcome such event and to resume performance of its obligation", {"entities":[(115,153,"Notification_type")]}),
("rieqpori nwr jrewjrketjkl en,mtn, m earthquake accident acts of the public enemy war terrorism rebellion insurrection riot and invasion.In the event of the Party affected shall promptly notify the other Party either by telephoneemailor any oral confirmation and shall use reasonable ddaf rqerqer qerqercommercial efforts to eliminate cure or overcome such event and to resume performance of its obligation", {"entities":[(219,257,"Notification_type")]}),
("Notwithstanding the foregoing should any event of Force Majeure as defined above prevent the Parties from performing the Agreement for a period exceeding three (3) months the nonaffected Party shall have the right to terminate this Agreement without further notice.It is expressly agreed by the Parties that strikes", {"entities":[(160,170,"Termination_period")]}),
("any event of Force Majeure as defined above prevent the Parties from performing the Agreement for a period exceeding three (3) months the nonaffected Party shall have the right to terminate this Agreement without further notice.It is expressly agreed by the Parties that strikes", {"entities":[(123,133,"Termination_period")]}),
("prevent the Parties from performing the Agreement for a period exceeding three (3) months the nonaffected Party shall have the right to terminate this Agreement without further notice.It is expressly agreed by the Parties that strikes", {"entities":[(79,89,"Termination_period")]}),
("Agreement for a period exceeding three (3) months the nonaffected Party shall have the right to terminate this Agreement without further notice", {"entities":[(39,49,"Termination_period")]}),
("j mrneq,mrqne ejrkeqrj ip emrn,rmq eevent of Force Majeure as defined above prevent the Parties from performing the Agreement for a period exceeding three (3) months the nonaffected Party shall have the right to terminate this Agreement without further notice. wkqr;jq rjeq eirpqeir rmneqmr, eiporeiIt is expressly agreed by the Parties that strikes", {"entities":[(155,165,"Termination_period")]}),
("Notwithstanding the foregoing should any event of rwrwtrwttrwtrw rwtrwtrw Force Majeure as defined above prevent the Parties from performing the Agreement for a period exceeding three (3) months the nonaffected Party shall have werewrew wtrwt vbdfghdfh  trwtwrthe right to terminate this Agreement without further notice.It is expressly agreed by the Parties that strikes", {"entities":[(184,194,"Termination_period")]}),
("wtrwtrw rwtrwtrwt rwtrtrwtrwt rwtrwtrwNotwithstanding the foregoing should any event of Force Majeure as defined above prevent the Parties from performing the Agreement for a period exceeding three (3) months the nonaffected Party shall have the right to terminate this Agreement ettrw wrtrwt hdgfh d", {"entities":[(198,208,"Termination_period")]}),
("rwtt rtrwtrw fhdgfhkj upipo ipoi m,n Parties from performing the Agreement for a period exceeding three (3) months the nonaffected Party shall have the right to terminate this Agreement without further notice.It is expressly agreed by the Parties that strikes rrqerqer eqrqere", {"entities":[(104,114,"Termination_period")]}),
("pioimbnreqnbrm wtyrhdkj brqewmnbr qwruiqo 12 qewjsajs qppor Notwithstanding the foregoing should any event of Force Majeure as defined above prevent the Parties from performing the Agreement for a period exceeding three (3) months the nonaffected Party shall have the right to terminate this Agreement without further notice.It is expressly agreed by the Parties that strikes", {"entities":[(220,230,"Termination_period")]}),
("Notwithstanding the  ereq ereq qerqer qepiopip opmbnmnb vvgfg nn foregoing should any event of Force Majeure as defined above prevent the Parties from performing the Agreement for a period exceeding three (3) months the nonaffected Party shall have the right to terminate this  kjklrjqewasbmnvgjh j ljk Agreement without further notice.It is expressly agreed by the Parties that strikes", {"entities":[(205,215,"Termination_period")]}),
("Force Majeure as defined above prevent the Parties from performing the Agreement for a period exceeding three (3) months the nonaffected Party shall have the right to terminate this Agreement without further notice.It is expressly agreed by the Parties that strikes wrettr fdshfhdh", {"entities":[(110,120,"Termination_period")]}),
("Notwithstanding the foregoing should any event of Force Majeure as defined above  wtrwt rwtrwt rwtrw fghfsgfsgmgjkpou jmhfj aew1 2 prevent the Parties from performing the Agreement for a period exceeding three (3) months the nonaffected Party shall have the right to terminate this Agreement without further notice.It is expressly agreed by the Parties that strikes", {"entities":[(210,220,"Termination_period")]}),
("The Contract Period shall be extended by a period of time equal to the period of interruption caused by an Event of Force Majeure.1.2.6 OR 1.2.7 7.2.8 All notices requests demands and other communications under the Force Majeure Event shall be  given by (a) telephone  (b) mailed by reputable overnight courier and receipted for by the party to whom said notice or other communication shall have been directed or (c) sent by facsimile transmission with receipt of oral confirmation that such transmission has been received", {"entities":[(258,279,"Notification_type")]}),
("period of time equal to the period of interruption caused by an Event of Force Majeure.1.2.6 OR 1.2.7 7.2.8 All notices requests demands and other communications under the Force Majeure Event shall be  given by (a) telephone  (b) mailed by reputable overnight courier and receipted for by the party to whom said notice or other communication shall have been directed ", {"entities":[(215,236,"Notification_type")]}),
("period of interruption caused by an Event of Force Majeure.1.2.6 OR 1.2.7 7.2.8 All notices requests demands and other communications under the Force Majeure Event shall be  given by (a) telephone  (b) mailed by reputable overnight courier and receipted for by the party to whom said notice or other communication shall have been directed or (c) sent by facsimile transmission with receipt of oral confirmation that such transmission has been received", {"entities":[(187,208,"Notification_type")]}),
("Event of Force Majeure.1.2.6 OR 1.2.7 7.2.8 All notices requests demands and other communications under the Force Majeure Event shall be  given by (a) telephone  (b) mailed by reputable overnight courier and receipted for by the party to whom said notice or other communication shall have been directed or (c) sent by facsimile transmission with receipt of oral confirmation that such transmission has been received", {"entities":[(151,172,"Notification_type")]}),
("All notices requests demands and other communications under the Force Majeure Event shall be  given by (a) telephone  (b) mailed by reputable overnight courier and receipted for by the party to whom said notice or other communication shall have been directed or (c) sent by facsimile transmission with receipt of oral confirmation that such transmission has been received", {"entities":[(107,128,"Notification_type")]}),
("kljkl kjwl ewieuiwqu brnemqb notices requests demands and other communications under the Force Majeure Event shall be  given by (a) telephone  (b) mailed by reputable overnight courier ", {"entities":[(132,153,"Notification_type")]}),
(" wrqerqe erqerqe qerqebmnb epoipoasb ewrejqi interruption caused by an Event of Force Majeure.1.2.6 OR 1.2.7 7.2.8 All notices requests demands and other communications under the Force Majeure Event shall be  given by (a) telephone  (b) mailed by reputable overnight courier and receipted for by the party to whom said notice or other communication shall have been directed or (c) sent by facsimile transmission with receipt of oral confirmation that such transmission has been received", {"entities":[(222,243,"Notification_type")]}),
(" ereqr eqreqr eqrqereq All notices requests demands and other communications under the Force Majeure Event shall be  given by (a) telephone  (b) mailed by reputable overnight courier and receipted for by the party to whom said notice or other  dfdaf dafdafdaf dafda ewqw communication shall have been directed or (c) sent by facsimile transmission with receipt of oral confirmation that such transmission has been received", {"entities":[(130,151,"Notification_type")]}),
(" ereq sdfgfs gsfdg sgfsinterruption caused by an Event of Force Majeure.1.2.6 OR 1.2.7 7.2.8 All notices requests demands and other communications under the Force Majeure Event shall be  given by (a) telephone  (b) mailed by reputable overnight courier and receipted for by the party ereq drqerqe mnwqbem woirpr", {"entities":[(200,221,"Notification_type")]}),
("The ereqrqer bm hp arw  Contract Period qerrqe shall be extended by a period of time equal to the period of interruption caused by an Event of Force Majeure.1.2.6 OR 1.2.7 7.2.8 All notices requests demands and other eqrqe fsggh eeqr communications under the Force Majeure Event shall be  given by (a) telephone  (b) mailed by reputable overnight courier and receipted for by the party to whom said notice or other communication shall have been directed or (c) sent by facsimile transmission with receipt of oral confirmation that such transmission has been received", {"entities":[(302,323,"Notification_type")]}),
("requests demands and other communications under the Force Majeure Event shall be  given by (a) telephone  (b) mailed by reputable overnight courier and receipted for by the party to whom said notice or other communication shall have been directed or (c) sent by facsimile transmission with receipt of oral confirmation", {"entities":[(95,116,"Notification_type")]}),
("Parties shall consult together in relation to the above matters following the occurrence of a Force Majeure Event.(4) and other communications hereunder shall be given in writing to the parties within 30 business days (5) Should paragraph (1) apply as a result of a single Force Majeure Event", {"entities":[(171,178,"Notification_type")]}),
("Parties wewew  grrw shall  ewe weweconsult together in relation to the above matters following the occurrence of a Force Majeure Event.(4) and other communications hereunder shall be given in writing to the parties within 30 business days ", {"entities":[(192,199,"Notification_type")]}),
(" wewq mnr,mqew rq ewo mmbm occurrence of a Force Majeure Event.(4) and other communications hereunder shall be given in writing to the parties within 30 business days (5) Should paragraph (1) apply as a result of a single Force Majeure Event", {"entities":[(120,127,"Notification_type")]}),
("Parties shall afwq pi  consult together in relation to the above matters following qwrqwr asfa  the occurrence of a Force Majeure Event.(4) and other communications hereunder shall be given in writing to the parties qwrrqe bmn ljqlw wpoeow bavbsvd kljklqew 213 eqrqr ", {"entities":[(193,200,"Notification_type")]}),
("matters following the occurrence of a Force Majeure Event.(4) and other communications hereunder shall be given in writing to the parties within 30 business days (5) Should paragraph (1) apply as a result of a single Force Majeure Event 4546 qweqwr em,n,mn qpipoi qbnb hjhqj 564 qwqsfdayghklj klmfbrwrifhbagqe ", {"entities":[(115,122,"Notification_type")]}),
("pioiaffdafqre nbfj lae pi wqjh jlParties shall consult together in relation to the above matters following the occurrence of a Force Majeure Event.(4) and other communications hereunder shall be given in writing to the parties within 30 business days (5) Should paragraph (1) apply as a result of a single Force Majeure Event", {"entities":[(204,211,"Notification_type")]}),
(" qwrqer sgskljl keirqeponasbbw p[qr 45 qrq together in relation to the above matters following the occurrence of a Force Majeure Event.(4) and other communications hereunder shall be given in writing to the parties within 30 business days 454 qrer b nmnrbqip qrjhja jewp wori mn,mqe", {"entities":[(192,199,"Notification_type")]}),
("pwio rfjqewhr bn,mb kljwrp ipor lj 121 31  and other communications hereunder shall be given in writing to the parties within 30 business days (5) Should paragraph (1) apply as a result of a single Force Majeure Event", {"entities":[(96,103,"Notification_type")]}),
("request, notices and other communications hereunder shall be given in writing to the parties within 30 business days (5) Should paragraph (1) apply as a result of a single Force Majeure Event", {"entities":[(70,77,"Notification_type")]}),
("Force Majeure Event.(4) and other communications hereunder shall be given in writing to the parties within 30 business days (5) Should paragraph (1) apply as a result of a single Force Majeure Event", {"entities":[(77,84,"Notification_type")]}),
(" wreqr n,mba qwie abhj eq[ lkj 3 wq qr occurrence of a Force Majeure Event.(4) and other communications hereunder shall be given in writing to the parties within 30 business days (5) Should paragraph (1) apply as a result of a single Force Majeure Event", {"entities":[(132,139,"Notification_type")]}),
("Should paragraph (1) apply as a result of a single Force Majeure Event for a continuous period of more than 60 days then the parties shall endeavor to agree any modifications to this Agreement (including without limitation determination of new tariffs (if appropriate) in accordance with the provisions of Clause 7(4)(e)) which may be equitable having regard to the nature of the Force Majeure Event", {"entities":[(108,110,"Termination_period")]}),
(" adas wreqwr rer result of a single Force Majeure Event for a continuous period of more than 60 days then the parties shall endeavor to agree any modifications to this Agreement (including without limitation determination of new tariffs (if appropriate) in accordance with the provisions of Clause 7(4)(e)) which may be equitable having regard to the nature of the Force Majeure Event", {"entities":[(93,95,"Termination_period")]}),
("Force Majeure Event for a continuous period of more than 60 days then the parties shall endeavor to agree any modifications to this Agreement (including without limitation determination of new tariffs (if appropriate) in accordance with the provisions of Clause 7(4)(e)) which may be equitable having regard to the nature of the Force Majeure Event", {"entities":[(57,59,"Termination_period")]}),
("qrer sgfsg bndhdqr qqrljl jqpoioqeri ennr Force Majeure Event for a continuous period of more than 60 days then the parties shall endeavor to agree any modifications to this Agreement (including without limitation determination of new tariffs", {"entities":[(99,101,"Termination_period")]}),
("klrjl erne,qpoiwepaskjhsshg dkseirei result of a single Force Majeure Event for a continuous period of more than 60 days then the parties shall endeavor to agree any modifications to this Agreement (including without limitation determination of new tariffs qrqer dfdfdaaew (if appropriate) in accordance with the provisions of Clause 7(4)(e)) which may be equitable having regard to the nature of the Force Majeure Event", {"entities":[(113,115,"Termination_period")]}),
("wrewq nbfnab iqpori mqn,l 123 qwrqerafa 356 single Force Majeure Event for a continuous period of more than 60 days then the parties shall endeavor to agree any modifications to this Agreement (including without limitation determination of new tariffs (if appropriate) in accordance with the provisions of Clause 7(4)(e)) which may be equitable having regard to the nature of the Force Majeure Event", {"entities":[(108,110,"Termination_period")]}),
("Should qrqer qrqerfbghjihd  paragraph qrqer qreqr  apply as a result of a single Force Majeure Event for a continuous period of more than 60 days then the parties shall endeavor to agree any modifications to this Agreement reqwr 45 qrereqr", {"entities":[(138,140,"Termination_period")]}),
(" wqr nfgrt rwi ntbrwkjtjsfreiwptdn kjapori  321 rtrwtg 5448 yt result of a single Force Majeure Event for a continuous period of more than 60 days then the parties shall endeavor to agree any modifications to this Agreement (including without limitation determination of new tariffs (if appropriate) in accordance with the provisions of Clause 7(4)(e)) which may be equitable having regard to the nature of the Force Majeure Event", {"entities":[(139,141,"Termination_period")]}),
("po yteyipo  qer apply as a result of a single Force Majeure Event for a continuous period of more than 60 days then the parties shall endeavor to agree any modifications to this Agreement (including without limitation determination of new tariffs (if appropriate) in accordance with the provisions", {"entities":[(103,105,"Termination_period")]}),
("121 qreqrhghj bght yiuiForce Majeure Event for a continuous period of more than 60 days then the parties shall endeavor to agree any modifications to this Agreement (including without limitation determination of new tariffs (if appropriate) in accordance with the provisions of Clause 7(4)(e)) which may be equitable having regard to the nature of the Force Majeure Event", {"entities":[(80,82,"Termination_period")]}),
("comply with all or a material part of its obligations under this Agreement(1) Neither Party shall be in breach of its obligations under this Agreement (other than payment obligations) or incur any liability to the other Party for any losses or damages of any nature whatsoever incurred or suffered", {"entities":[(78,91,"Liability")]}),
("part of its obligations under this Agreement(1) Neither Party shall jklejkl mn,mnm be in breach of its obligations under this Agreement (other than payment obligations) or incur any liability to the other Party for any losses or damages of any nature whatsoever incurred or suffered", {"entities":[(48,61,"Liability")]}),
("under this Agreement(1) Neither Party shall be in breach of its obligations under this Agreement (other than payment obligations) or incur any liability to the other Party for any losses or damages of any nature whatsoever incurred or suffered rerqer fggfsg", {"entities":[(24,37,"Liability")]}),
("qwieuqiwu mn,mrn bnebr kljkljq[[ comply with all or a material part of its obligations under this Agreement(1) Neither Party shall be in breach of its obligations under this Agreement", {"entities":[(111,124,"Liability")]}),
("rejhfjpo kt[ rwmdnfr qj nrqerjj o rwt231 2898 9 Neither Party shall be in breach of its obligations under this Agreement (other than payment obligations) or incur any liability to the other Party for any losses or damages of any nature whatsoever incurred or suffered", {"entities":[(48,61,"Liability")]}),
("12132 98 eqreqr dghjkpp ghjgfyt drere t3qejjrty vnmvbcz comply with all or a material part of its obligations under this Agreement(1) Neither Party shall be in breach of its obligations under this Agreement (other than payment obligations) or incur any liability to the other Party for any losses or damages of any nature whatsoever incurred or suffered", {"entities":[(134,147,"Liability")]}),
("under this Agreement aererq gh korgxcvbzv ztr ta (1) Neither Party shall be in breach of its obligations under this Agreement (other than payment obligations) or incur any liability to the other Party for any losses or damages of any nature whatsoever incurred or suffered", {"entities":[(53,66,"Liability")]}),
("comply with all or a material part of ereqrdzss n  ufi z gesr e  its obligations under this Agreement(1) Neither Party shall be in breach of its obligations under this Agreement (other than payment obligations) aerae ry8 jc 64yu4 rtyu ", {"entities":[(105,118,"Liability")]}),
("matters following the occurrence of a Force Majeure Event.(4) The nonperforming Party shall notify the other Party of such force majeure within ten (10) days after such occurrence by giving written notice to the other Party stating the nature of the event its anticipated duration and any action being taken to avoid or minimize its effect", {"entities":[(190,197,"Notification_type")]}),
("occurrence of a Force Majeure Event.(4) The nonperforming Party shall notify the other Party of such force majeure within ten (10) days after such occurrence by giving written notice to the other Party stating the nature of the event its anticipated duration", {"entities":[(168,175,"Notification_type")]}),
("The nonperforming Party shall notify the other Party of such force majeure within ten (10) days after such occurrence by giving written notice to the other Party stating the nature of the event its anticipated duration and any action", {"entities":[(128,135,"Notification_type")]}),
("Party shall notify the other Party of such force majeure within ten (10) days after such occurrence by giving written notice to the other Party stating the nature of the event its anticipated duration and any action being taken to avoid or minimize its effect", {"entities":[(110,117,"Notification_type")]}),
("matters following the occurrence of a Force Majeure Event. Party shall notify the other Party of such force majeure within ten (10) days after such occurrence by giving written notice to the other Party stating the nature", {"entities":[(169,176,"Notification_type")]}),
("oprfsjgh sdnb azdnnwqire dkfjlkjekj 5658 qwrvf matters following the occurrence of a Force Majeure Event.(4) The nonperforming Party shall notify the other Party of such force majeure within ten (10) days after such occurrence by giving written notice to the other Party stating the nature of the event its anticipated duration and any action being taken to avoid or minimize its effect", {"entities":[(237,244,"Notification_type")]}),
(" arert dfvghthjt wq rhuyioiygh df notify the other Party of such force majeure within ten (10) days after such occurrence by giving written notice to the other Party stating the nature of the event its anticipated duration and any action being taken to avoid or minimize its effect", {"entities":[(132,139,"Notification_type")]}),
("qrerdfdnmgjko xxbvxb scvc wrrqer qererThe nonperforming Party shall notify the other Party of such force majeure within ten (10) days after such occurrence by giving written notice to the other Party stating the nature of the event its anticipated duration rtrw rewghjys dfgtasr 546 r4atrt ra", {"entities":[(166,173,"Notification_type")]}),
("lkjgfskl dmnd,manrfar ueu sdfg Party shall notify the other Party of such force majeure within ten (10) days after such occurrence by giving written notice to the other Party stating the nature of the event its anticipated duration and any action being taken to avoid or minimize its effect dcxbthy yuyuxfdrf ", {"entities":[(141,148,"Notification_type")]}),
("matters following the occurrence of a Force Majeure Event.(4) sdgfsgrw vxbvnrt wevx fsg The nonperforming Party shall notify the other Party of such force majeure within ten (10) days after such occurrence by giving written notice to the other Party stating the nature of the event its anticipated duration and any action being taken to avoid or minimize its effect", {"entities":[(216,223,"Notification_type")]}),
("to be a change in the Services andor Deliverables or entitle Vendor to any type of equitable adjustment to the fees.  7. FORCE MAJEURE.  7.1.1 No Party shall be deemed in default of this Agreement or unless otherwise expressly provided therein any Ancillary Agreement for any delay or failure to fulfill any obligation", {"entities":[(143,151,"Liability")]}),
(" Vendor to any type of equitable adjustment to the fees.  7. FORCE MAJEURE.  7.1.1 No Party shall be deemed in default of this Agreement or unless otherwise expressly provided therein any Ancillary Agreement for any delay or failure to fulfill any obligation wrpoiddsg hertkjrh ehnwhe 4 wrqererq", {"entities":[(83,91,"Liability")]}),
("FORCE MAJEURE.  7.1.1 No Party shall be deemed in default of this Agreement or unless otherwise expressly provided therein any Ancillary Agreement for any delay or failure to fulfill any obligation", {"entities":[(22,30,"Liability")]}),
("No Party shall be deemed in default of this Agreement or unless otherwise expressly provided therein any Ancillary Agreement for any delay or failure to fulfill any obligation", {"entities":[(0,8,"Liability")]}),
("Deliverables or entitle Vendor to any type of equitable adjustment to the fees.  7. FORCE MAJEURE.  7.1.1 No Party shall be deemed in default of this Agreement or unless otherwise expressly provided therein any  wewepwo mqenrqm p[woei xfbkjasrhqwrq eroriui Ancillary Agreement for any delay or failure to fulfill any obligation ", {"entities":[(106,114,"Liability")]}),
("wr dngfsmgn ei kljafklj adjustment to the fees.  7. FORCE MAJEURE.  7.1.1 No Party shall be deemed in default of this Agreement or unless otherwise expressly provided therein any Ancillary Agreement for any delay or failure to fulfill any obligation", {"entities":[(74,82,"Liability")]}),
("to be a change in the Services andor Deliverables or entitle Vendor  wipo nf iqeror ereqjjals peri rtarmjbtn to any type of equitable adjustment to the fees.  7. FORCE MAJEURE.  7.1.1 No Party shall be deemed in default of this Agreement or unless otherwise expressly provided therein any Ancillary Agreement ", {"entities":[(184,192,"Liability")]}),
("er ernelr pir fngkljner aewi mnrmenr equitable adjustment to the fees.  7. FORCE MAJEURE.  7.1.1 No Party shall be deemed in default of this Agreement or unless otherwise expressly provided therein any Ancillary Agreement for any delay or failure to fulfill  qr pi eqmnr,m opipor pori enrejany obligation", {"entities":[(97,105,"Liability")]}),
("eirqueir m,nafa lpipo eq erjh brbqnermn ajkffl No Party shall be deemed in default of this Agreement or unless otherwise expressly provided therein any Ancillary Agreement for any delay or failure to fulfill any obligation", {"entities":[(47,55,"Liability")]}),
("poripo dfb eqr hqe mnadbfnbd 112 erqer FORCE MAJEURE.  7.1.1 No Party shall be deemed in default of this Agreement or unless otherwise expressly provided therein any Ancillary Agreement for any delay or failure to fulfill any obligation qere vhng uujk sdte", {"entities":[(61,69,"Liability")]}),
("Services andor Deliverables or entitle Vendor to any type of equitable adjustment to the fees.  7. FORCE MAJEURE.  7.1.1 No Party shall be deemed in default of this Agreement or unless otherwise expressly provided therein any Ancillary Agreement  poqiw ofnlka qwjkl rfor any delay or failure to fulfill any obligation", {"entities":[(121,129,"Liability")]}),
("reasonable efforts to minimize the duration of any force majeure.Force Majeure Event affects the performance of the claiming Party for 180 consecutive days the nonclaiming Party may terminate this Agreement.Schedule", {"entities":[(135,138,"Termination_period")]}),
("minimize the duration of any force majeure.Force Majeure Event affects the performance of the claiming Party for 180 consecutive days the nonclaiming Party may terminate this Agreement.Schedule", {"entities":[(113,116,"Termination_period")]}),
("Force Majeure Event affects the performance of the claiming Party for 180 consecutive days the nonclaiming Party may terminate this Agreement.Schedule", {"entities":[(70,73,"Termination_period")]}),
("reasonable efforts to minimize the duration of any wqrqr vb dj upo force majeure.Force Majeure Event affects the performance of the claiming Party for 180 consecutive days the nonclaiming Party may terminate this Agreement.Schedule", {"entities":[(151,154,"Termination_period")]}),
(" pirpqor emnr jerkle reptiop[ fsgmnl Force Majeure Event affects the performance of the claiming Party for 180 consecutive days the nonclaiming Party may terminate this Agreement.Schedule", {"entities":[(107,110,"Termination_period")]}),
("efforts to minimize the duration of any force majeure.Force Majeure Event affects the performance of the claiming Party for 180 consecutive days the nonclaiming Party may terminate this Agreement arporqi mna, qr;lqkr kq[er qeiurqe rup ", {"entities":[(124,127,"Termination_period")]}),
("poirqer dmnfd gkjqla qw peripqe 34 jkra preipo Force Majeure Event affects the performance of the claiming Party for 180 consecutive days the nonclaiming Party may terminate this Agreement.Schedule", {"entities":[(117,120,"Termination_period")]}),
("ri pdgndg jaerpqei ehjkleh rpeqriop eqrhlkj affects the performance of the claiming Party for 180 consecutive days the nonclaiming Party may terminate this Agreement.Schedule peir dabfkjadhrf", {"entities":[(94,97,"Termination_period")]}),
("poirpo ,mtnm dgjdfksgj eoqiro[eri Force Majeure Event affects the performance of the claiming Party for 180 consecutive days the nonclaiming Party may terminate this porioq dkjafdjah qorqepor m,anf", {"entities":[(104,107,"Termination_period")]}),
(" eriq dmngfdngf ;ar;ejkj jrfa eiurprio Event affects the performance of the claiming Party for 180 consecutive days the nonclaiming Party may terminate this Agreement.Schedule", {"entities":[(95,98,"Termination_period")]}),
("reasonable efforts to etrew vb hngj ryru sgt minimize the duration of any force majeure.Force Majeure Event affects the performance of the claiming Party for 180 consecutive days the nonclaiming Party may terminate this Agreement.Schedule erert hg gj req jf uiyp  ffg", {"entities":[(158,161,"Termination_period")]}),
("FORCE MAJEURE. No party will be held responsible to the other Party nor be deemed to be in default under or in breach of any provision of this Agreement for failure or delay in performing any obligation of this Agreement", {"entities":[(15,23,"Liability")]}),
("No party will be held responsible to the other Party nor be deemed to be in default under or in breach of any provision of this Agreement for failure or delay in performing any obligation of this Agreement rewr qqe poiorq ", {"entities":[(0,8,"Liability")]}),
("poio dflkj adnf,mnFORCE MAJEURE. No party will be held responsible to the other Party nor be deemed to be in default under or in breach of any provision of this Agreement for failure or delay in performing any obligation of this Agreement", {"entities":[(33,41,"Liability")]}),
("poeipoqw menr,mn dfjdpafiq poirqeirpo No party will be held responsible to the other Party nor be deemed to be in default under or in breach of any provision of this Agreement for failure or delay in performing any obligation of this Agreement", {"entities":[(38,46,"Liability")]}),
("poripor dm,nf,dmnf lahjskljare qwrpqwro djfdklgfj FORCE MAJEURE. No party will be held responsible to the other Party nor be deemed to be in default under or in breach of any provision of this Agreement for failure or delay in performing any obligation of this Agreement", {"entities":[(65,73,"Liability")]}),
("gfhdaf pripo vmncsnfgds sajkldj No party will be held responsible to the other Party nor be deemed to be in default under or in breach of any provision of this Agreement for failure or delay in performing any obligation of this Agreement", {"entities":[(32,40,"Liability")]}),
("fdgasgwe ofiodppgf mfsgl 12 rerep sfkd;;fl potipor klsgjklsg FORCE MAJEURE. No party will be held responsible to the other Party nor be deemed to be in default under or in breach of any provision of this Agreement for failure or delay in performing any obligation of this pori dlafjkljpoewripori Agreement", {"entities":[(76,84,"Liability")]}),
("pierpo dkljklg jbmnbm hgdhjf mnbgdfsnbg klj No party will be held responsible to the other Party nor be deemed to be in default optipor dklsjklg eoio dklskljgskl under or in breach of any provision of this Agreement for failure or delay in performing any obligation of this Agreement", {"entities":[(44,52,"Liability")]}),
("dfhgyew afdhdjafhe q eirpeoio po rnvmfsgndkkjgf No party will be held responsible to the other Party nor be deemed to be in default under or in breach of any provision of this Agreement for failure or delay in performing any obligation of this Agreement", {"entities":[(48,56,"Liability")]}),
("irpeor fkgjkgjfl gnf,gn, jrejri oiwo rqworip dslgjsdfg kjj;lrtop jfggkj tiporti FORCE MAJEURE. No party will be held responsible to the other Party nor be deemed to be in default under or in breach of any provision of this Agreement for failure or delay in performing any obligation of this Agreement", {"entities":[(95,103,"Liability")]}),
("12 qerqr scfgsg FORCE MAJEURE. No party will be held responsible to the other Party nor be deemed to be in default under or in breach of any provision of this Agreement for failure or delay in performing any obligation of this Agreement dafoipdaf qeiru brqaebr j", {"entities":[(31,39,"Liability")]}),
("All notices requests demands and other communications with respect to Force Majeure Events hereunder shall be in writing and shall be deemed to have been given (i) when delivered by hand or (ii) if mailed by certified or registered mail with postage", {"entities":[(113,120,"Notification_type")]}),
("requests demands and other communications with respect to Force Majeure Events hereunder shall be in writing and shall be deemed to have been given (i) when delivered by hand or (ii) if mailed by certified or registered mail with postage", {"entities":[(101,108,"Notification_type")]}),
("demands and other communications with respect to Force Majeure Events hereunder shall be in writing and shall be deemed to have been given (i) when delivered by hand or (ii) if mailed by certified or registered mail with postage", {"entities":[(92,99,"Notification_type")]}),
("and other communications with respect to Force Majeure Events hereunder shall be in writing and shall be deemed to have been given (i) when delivered by hand or (ii) if mailed by certified or registered mail with postage", {"entities":[(84,91,"Notification_type")]}),
("poirpoq dnfdlaf kljwkjre asljsafk kljkalfjAll notices requests demands and other communications with respect to Force Majeure Events hereunder shall be in writing and shall be deemed to have been given (i) when delivered by hand or (ii) if mailed by certified or registered mail with postage", {"entities":[(155,162,"Notification_type")]}),
("All notices requests  e[ipor kafjfkl afpowiqpor a ffj msfn,mfandemands and other communications with respect to Force Majeure Events hereunder shall be in writing and shall be deemed to have been given (i) when delivered by hand or (ii) if mailed by certified or registered mail with postage eiqrp odkljfkl qwri ", {"entities":[(155,162,"Notification_type")]}),
("ghfhda ajkew poriqop dafkjh asf,maf kljkAll notices requests demands and other communications with respect to Force Majeure Events hereunder shall be in writing and shall be deemed to have been given (i) when delivered by hand or (ii) if mailed by certified or registered mail with postage", {"entities":[(153,160,"Notification_type")]}),
("All notices requests demands and other communications with respect to Force Majeure Events hereunder shall be in writing and shall be deemed to have been given (i) when delivered by hand or (poriqpo hagfhj abnqewbr qwekqjwe asdpai qjwhrqjrh", {"entities":[(113,120,"Notification_type")]}),
("fghagh ahjsklj All notices requests demands and other communications with respect to Force Majeure Events hereunder shall be in writing and shall be deemed to have been given (i) when delivered by hand or (ii) if mailed by certified or registered mail with postage", {"entities":[(128,135,"Notification_type")]}),
("kjsghghasf qwuioqw jhafj mafbn, and other communications with respect to Force Majeure Events hereunder shall be in writing and shall be deemed to have been given (i) when delivered by hand or (ii) if mailed by certified or registered mail with postage", {"entities":[(116,123,"Notification_type")]}),
("hfaga wui kasfjklf and other communications with respect to Force Majeure Events hereunder shall be in writing and shall be deemed to have been given (i) when delivered by hand or (ii) if mailed by certified or registered mail with postage", {"entities":[(103,110,"Notification_type")]}),
("Force Majeure Event for the performance of its obligations.If a Force Majeure Event affects the performance of the claiming Party for sixty (60) consecutive days the nonclaiming Party may terminate this Agreement", {"entities":[(141,143,"Termination_period")]}),
("for the performance of its obligations.If a Force Majeure Event affects the performance of the claiming Party for sixty (60) consecutive days the nonclaiming Party may terminate this Agreement", {"entities":[(121,123,"Termination_period")]}),
("If a Force Majeure Event affects the performance of the claiming Party for sixty (60) consecutive days the nonclaiming Party may terminate this Agreement", {"entities":[(82,84,"Termination_period")]}),
("qeuwte dklfjdkjf  iopo dnafnf jklfka Force Majeure Event affects the performance of the claiming Party for sixty (60) consecutive days the nonclaiming Party may terminate this Agreement", {"entities":[(114,116,"Termination_period")]}),
("sdbkaj nvdafjkl lk poipor ashfkIf a Force Majeure Event affects the performance of the claiming Party for sixty (60) consecutive days the nonclaiming Party may terminate this Agreement sadflj wqriui poakfj;la ", {"entities":[(113,115,"Termination_period")]}),
("kjdhjf irueori ryuqeori djfhdjhf Force Majeure Event affects the performance of the claiming Party for sixty (60) consecutive days the nonclaiming Party may terminate this Agreement", {"entities":[(110,112,"Termination_period")]}),
("orq dklfj oeiwpo lafjflf nbfdamnbf lkjr affects the performance of the claiming Party for sixty (60) consecutive days the nonclaiming Party may terminate this Agreement", {"entities":[(97,99,"Termination_period")]}),
("performance of its obligations.If a Force Majeure Event affects the performance of the claiming Party for sixty (60) consecutive days the nonclaiming Party may terminate this Agreement", {"entities":[(113,115,"Termination_period")]}),
("jgdhjg eiru rklvjdklgfj Force Majeure Event for the performance of its obligations.If a Force Majeure Event affects the performance of the claiming Party for sixty (60) consecutive days the uer dklfjkl nonclaiming Party may terminate this Agreement", {"entities":[(165,167,"Termination_period")]}),
("Force Majeure Event klfjda poruieq ofjkl nd,mnfamfor the performance of its obligations.If a Force Majeure Event affects the performance of the claiming Party for sixty (60) consecutive days the nonclaiming Party may terminate this Agreement", {"entities":[(170,172,"Termination_period")]}),
("Force Majeure Event for the performance aprio kjdhfda feuryue  of its obligations.If a Force Majeure Event affects the performance of the claiming Party for sixty (60) consecutive days the nonclaiming Party may terminate this Agreement  iur idhfda woqripo", {"entities":[(164,166,"Termination_period")]}),
("If the occurrence of an event of force majeure causes a Party to fail or delay the performance of its contractual obligations for a period of two (60) consecutive months or longer the other Party may terminate this Agreement by written notice to the Party failing or delaying performance", {"entities":[(147,149,"Termination_period")]}),
("force majeure causes a Party to fail or delay the performance of its contractual obligations for a period of two (60) consecutive months or longer the other Party may terminate this Agreement by written notice to the Party failing or delaying performance", {"entities":[(114,116,"Termination_period")]}),
("occurrence of an event of force majeure causes a Party to fail or delay the performance of its contractual obligations for a period of two (60) consecutive months or longer the other Party may terminate this Agreement by written notice to the Party failing or delaying performance", {"entities":[(140,142,"Termination_period")]}),
("If the occurrence of an event iouw kjdahf aqrhkj of force majeure causes a Party to fail or delay the performance of its contractual obligations for a period of two (60) consecutive months or longer the other Party may terminate this Agreement by written notice to the Party failing or delaying performance", {"entities":[(166,168,"Termination_period")]}),
("If the occurrence of an event of qwr aswq ewqr hnghjt ut  force majeure causes a Party to fail or delay the performance of its contractual obligations for a period of two (60) consecutive months or longer the other Party may terminate this Agreement by written notice to the Party failing or delaying performance", {"entities":[(172,174,"Termination_period")]}),
("event of force majeure causes a Party to fail or delay the performance of its contractual obligations for a period of two (60) consecutive months or longer the other Party may terminate this Agreement by written notice to the Party failing or delaying performance", {"entities":[(123,125,"Termination_period")]}),
("poriq okljhakj quewryqi kj force majeure causes a Party to fail or delay the performance of its contractual obligations for a period of two (60) consecutive months or longer the other Party may terminate this Agreement by written notice to the Party failing or delaying performance", {"entities":[(141,143,"Termination_period")]}),
("iop wkjhf rq uriu causes a Party to fail or delay the performance of its contractual obligations for a period of two (60) consecutive months or longer the other Party may terminate this Agreement by written notice to the Party failing or delaying performance", {"entities":[(118,120,"Termination_period")]}),
("uyr kjahf iorui oasfido qwrqer 1 rwrqer adsf If the occurrence of an event of force majeure causes a Party to fail or delay the performance of its contractual obligations for a period of two (60) consecutive months or longer the other Party may terminate this Agreement by written notice to the Party failing or delaying performance", {"entities":[(192,194,"Termination_period")]}),
("irque dkjhfak If the occurrence of an event of force majeure causes a Party to fail or delay the performance of its contractual obligations for a period of two (60) consecutive months or longer the other Party may terminate this Agreement by written notice to the Party failing or delaying performance", {"entities":[(161,163,"Termination_period")]}),
("yur dkjh hfjd force majeure causes a Party to fail or delay the performance of its contractual obligations for a period of two (60) consecutive months or longer the other Party may terminate this Agreement by written notice qwer dsgfsdg to the Party failing or delaying performance rqewr fscgfsg", {"entities":[(128,130,"Termination_period")]}),
("Force Majeure must provide notice to the other Party. Initial notice may be given orally however written notification with reasonably full particulars of the event or occurrence is required as soon as reasonably possible. 7.1.3 A Party claiming Force Majeure will diligently use all reasonable efforts to remove the cause condition event or circumstance of such Force Majeure will promptly give written notice to the other Party of the termination of such Force Majeure and will resume performance of any suspended obligation as soon as reasonably possible after termination of such Force Majeure.", {"entities":[(97,104,"Notification_type")]}),
("notice to the other Party. Initial notice may be given orally however written notification with reasonably full particulars of the event or occurrence is required as soon as reasonably possible. 7.1.3 A Party claiming Force Majeure will diligently use all reasonable efforts to remove the cause condition event or circumstance of such Force Majeure will promptly give written notice to the other Party of the termination of such Force Majeure and will resume performance of any suspended obligation as soon as reasonably possible after termination of such Force Majeure.", {"entities":[(70,77,"Notification_type")]}),
("unable in whole or in part to carry out its obligations under this Agreement due to Force Majeure must provide notice to the other Party. Initial notice may be given orally however written notification with reasonably full particulars of the event or occurrence is required as soon as reasonably possible", {"entities":[(181,188,"Notification_type")]}),
("Agreement due to Force Majeure must provide notice to the other Party. Initial notice may be given orally however written notification with reasonably full particulars of the event or occurrence is required as soon as reasonably possible. 7.1.3 A Party claiming Force Majeure will diligently", {"entities":[(114,121,"Notification_type")]}),
("Initial notice may be given orally however written notification with reasonably full particulars of the event or occurrence is required as soon as reasonably possible", {"entities":[(43,50,"Notification_type")]}),
("obligations under this Agreement due to Force Majeure must provide notice to the other Party. Initial notice may be given orally however written notification with reasonably full particulars of the event or occurrence is required as soon as reasonably possible. 7.1.3 A Party claiming Force Majeure will diligently ", {"entities":[(137,144,"Notification_type")]}),
("poir dkjgfdhjgkh qeiu ashnjhr Party which is unable in whole or in part to carry out its obligations under this Agreement due to Force Majeure must provide notice to the other Party. Initial notice may be given orally however written notification with reasonably full particulars of the event or occurrence is required as soon as reasonably possible", {"entities":[(226,233,"Notification_type")]}),
("piwr dahfjahf wqreuio adf,mhhj notice to the other Party. Initial notice may be given orally however written notification with reasonably full particulars of the event or occurrence is required as soon as reasonably possible. 7.1.3 A Party claiming Force Majeure will diligently use all reasonable efforts to remove the cause condition event or circumstance of such Force Majeure ", {"entities":[(101,108,"Notification_type")]}),
("owqei dkjgfk wew kljekljqri eklj  Force Majeure must provide notice to the other Party. Initial notice may be given orally however written notification with reasonably full particulars of the event or occurrence is required as soon as reasonably possible. 7.1.3 A Party claiming Force Majeure will diligently use all reasonable efforts to remove", {"entities":[(131,138,"Notification_type")]}),
("ahfjl eqpri adkfj pworp A Party which is unable in whole or in part to carry out its obligations under this Agreement due to Force Majeure must provide notice to the other Party. Initial notice may be given orally however written notification with reasonably full particulars of the event or occurrence is required as soon as reasonably possible. 7.1.3 A Party claiming Force Majeure will diligently use all reasonable efforts to remove the cause condition event or circumstance of such Force Majeure", {"entities":[(222,229,"Notification_type")]}),
("uir kafjl riwopqw ahfklj Agreement due to Force Majeure must provide notice to the other Party. Initial notice may be given orally however written notification with reasonably full particulars of the event or occurrence is required as soon as reasonably possible. 7.1.3 A Party claiming Force Majeure will diligently use all reasonable efforts to remove the cause condition event or circumstance of such Force Majeure will promptly give written notice to the other Party ", {"entities":[(139,146,"Notification_type")]}),
("if an Event of Force Majeure occurs and its effect continues for a period of 180 days either the Authority or the Operator may give to the other a notice of termination.If Authority is paying fee during Force Majeure", {"entities":[(77,80,"Termination_period")]}),
("its effect continues for a period of 180 days either the Authority or the Operator may give to the other a notice of termination.If Authority is paying fee during Force Majeure", {"entities":[(37,40,"Termination_period")]}),
("hjafg qiuri ajsfklfjl Force Majeure occurs and its effect continues for a period of 180 days either the Authority or the Operator may give to the other a notice of termination.If Authority is paying fee during Force Majeure", {"entities":[(84,87,"Termination_period")]}),
("Event of Force Majeure occurs and its effect continues for a period of 180 days either the Authority or the Operator may give to the other a notice of termination.If Authority is paying fee during Force Majeure", {"entities":[(71,74,"Termination_period")]}),
("urewqi afjskl woiop  occurs and its effect continues for a period of 180 days either the Authority or the Operator may give to the other a notice of termination.If Authority is paying fee during Force Majeure oriopw klrjklerj ", {"entities":[(69,72,"Termination_period")]}),
("poqri akdfjkl oqriewo alfjkl Force Majeure occurs and its effect continues for a period of 180 days either the Authority or the Operator may give to the other a notice of termination.If Authority is paying fee during Force Majeure", {"entities":[(91,94,"Termination_period")]}),
("irpo kajfk oiwroasfkh hslrewiu its effect continues for a period of 180 days either the Authority or the Operator may give to the other a notice of termination.If Authority is paying fee during Force Majeure", {"entities":[(68,71,"Termination_period")]}),
("ryu jahfjkl if an Event of Force Majeure occurs and its effect continues for a period of 180 days either the Authority or the Operator may give to the other a notice of termination poriw sahjriweuoiafj asrqewr ", {"entities":[(89,92,"Termination_period")]}),
("irque jadfhkljhf if an Event of Force Majeure occurs and its effect continues for a period of 180 days either the Authority or the Operator may give to the other a notice of termination.If Authority is paying fee during Force Majeure", {"entities":[(94,97,"Termination_period")]}),
("piqor adfjkl eirurue Force Majeure occurs and its effect continues for a period of 180 days either the Authority or the Operator may give to the other a notice of termination.If Authority is paying fee during Force Majeure", {"entities":[(83,86,"Termination_period")]}),
("erqui hfjaklj urip if an Event of Force Majeure occurs and its effect continues for a period of 180 days either the Authority or the Operator may give to the other a notice of termination.If Authority is paying fee during Force Majeure", {"entities":[(96,99,"Termination_period")]}),
("Should paragraph (1) apply as a result of a single Force Majeure Event for a continuous period of more than 90 days then the parties shall endeavor to agree any modifications to this Agreement (including without limitation determination of new tariffs (if appropriate) in accordance with the provisions of Clause", {"entities":[(108,110,"Termination_period")]}),
("Should paragraph (1) apply as a result of a single Force Majeure Event for a continuous period of more than 60 days then the parties shall endeavor to agree any modifications to this Agreement (including without limitation determination of new tariffs (if appropriate) in accordance with the provisions of Clause", {"entities":[(108,110,"Termination_period")]}),
("Should paragraph (1) apply as a result of a single Force Majeure Event for a continuous period of more than 90 days then the parties shall endeavor to agree any modifications to this Agreement (including without limitation determination of new tariffs (if appropriate) in accordance with the provisions of Clause", {"entities":[(108,110,"Termination_period")]}),
("as a result of a single Force Majeure Event for a continuous period of more than 90 days then the parties shall endeavor to agree any modifications to this Agreement (including without limitation determination of new tariffs (if appropriate) in accordance with the provisions of Clause", {"entities":[(81,83,"Termination_period")]}),
("single Force Majeure Event for a continuous period of more than 90 days then the parties shall endeavor to agree any modifications to this Agreement (including without limitation determination of new tariffs (if appropriate) in accordance with the provisions of Clause", {"entities":[(64,66,"Termination_period")]}),
("iirquew dhjaf qiurio Force Majeure Event for a continuous period of more than 90 days then the parties shall endeavor to agree any modifications to this Agreement (including without limitation determination of new tariffs (if appropriate) in accordance with the provisions of Clause", {"entities":[(78,80,"Termination_period")]}),
("quw rjahf wqurooi asfh l asfa Force Majeure Event for a continuous period of more than 90 days then the parties shall endeavor to agree any modifications to this Agreement (including without limitation determination of new tariffs (if appropriate) in accordance with the provisions of Clause", {"entities":[(87,89,"Termination_period")]}),
("apply as a result of a single Force Majeure Event for a continuous period of more than 60 days then the parties shall endeavor to agree any modifications to this Agreement (including without limitation determination of new tariffs (if appropriate) in accordance with the provisions of Clause", {"entities":[(87,89,"Termination_period")]}),
("eqr fghfs single Force Majeure Event for a continuous period of more than 60 days then the parties shall endeavor to agree any modifications to this Agreement (including without limitation determination of new tariffs (if appropriate) in accordance with the provisions of Clause", {"entities":[(74,76,"Termination_period")]}),
("qerqe fhteytey single Force Majeure Event for a continuous period of more than 90 days then the parties shall endeavor to agree any modifications to this Agreement (including without limitation determination of new tariffs (if appropriate) in accordance with the provisions of Clause", {"entities":[(79,81,"Termination_period")]}),
("quiri podhja ladf single Force Majeure Event for a continuous period of more than 90 days then the parties shall endeavor to agree any modifications to this Agreement (including without limitation determination of new tariffs (if appropriate) in accordance with the provisions of Clause", {"entities":[(82,84,"Termination_period")]}),
("oir klfj lfkjda fdfd result of a single Force Majeure Event for a continuous period of more than 90 days then the parties shall endeavor to agree any modifications to this Agreement (including without limitation determination of new tariffs (if appropriate) in accordance with the provisions of Clause", {"entities":[(97,99,"Termination_period")]}),
]

In [45]:
#nlp = spacy.load(model_path) 
nlp = spacy.blank("en")
#nlp = spacy.load("en_core_web_md")

if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe("ner")
    # otherwise, get it, so we can add labels to it
else:
    ner = nlp.get_pipe("ner")

for LABEL in LABELS:
    ner.add_label(LABEL) 

In [46]:
from spacy.training.example import Example

optimizer = nlp.begin_training()
move_names = list(ner.move_names)
# get names of other pipes to disable them during training
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

n_iter=7

# only train NER
with nlp.disable_pipes(*other_pipes), warnings.catch_warnings():
    # show warnings for misaligned entity spans once
    warnings.filterwarnings("once", category=UserWarning, module='spacy')

    sizes = compounding(1.0, 4.0, 1.001)
    # batch up the examples using spaCy's minibatch
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        batches = minibatch(TRAIN_DATA, size=sizes)
        losses = {}
        for batch in batches:
            texts, annotations = zip(*batch)
            
            example = []
            # Update the model with iterating each text
            for i in range(len(texts)):
                doc = nlp.make_doc(texts[i])
                example.append(Example.from_dict(doc, annotations[i]))
            
            # Update the model
            nlp.update(example, drop=0.5, losses=losses)
        print("Losses", losses)

C:\Users\Harika\AppData\Local\Programs\Python\Python37\lib\site-packages\spacy\training\iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "In this Clause   Event of Force Majeure means an e..." with entities "[(228, 241, 'Liability')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,


Losses {'ner': 969.973376426401}
Losses {'ner': 385.49800922098996}
Losses {'ner': 442.60968948121825}
Losses {'ner': 211.08514829018165}
Losses {'ner': 153.49692451660107}
Losses {'ner': 96.3662896537201}
Losses {'ner': 99.70823149944317}


In [47]:
test_text = "SCHEDULE D FORM OF STATEMENT OF WORKStatement of Work Number PREAMBLEThis Statement of Work (SOW) effective as of 01st June 2019 (the SOW Effective Date) is entered into by and between AGI Corporation and Latentview Analytics Corporation  (Vendor) pursuant to the terms and conditions set forth in the Master Agreement.WHEREAS AGI Corporation issued and Vendor responded to a request for proposal (RFP) entitled) Disability RFPRFI Inquiry SelfService to which Vendor affirmatively responded and stated that it is fully capable and willing to provide Business intelligence implementation services services as more fully described in this SOW (the Services).WHEREAS AGI Corporation would like Vendor to provide and Vendor would like to provide the services as more fully described in this SOW (the Services). NOW THEREFORE in consideration of the foregoing and the mutual promises covenants and agreements set forth below and in the Master Agreement and for other good and valuable consideration the receipt and sufficiency of which the Parties hereby acknowledge the Parties agree as followsDEFINITIONS AND INTERPRETATIONS.Capitalized terms used but not defined herein shall have the meaning ascribed to them in the Master Agreement.PRECEDENCE.In the event of a conflict between the Master Agreement and this Statement of Work the Master Agreement shall take precedence in every instance unless this Statement of Work (i) expressly states that the parties desired to amend the Master Agreement (ii) specifically states the section of the Master Agreement to be amended and (iii) is executed by an authorized signatory of each party.  In connection therewith the parties hereby wish to amend certain legal terms of the Master Agreement to address local compliance with Law requirements as set forth on Addendum A annexed hereto.1.SERVICES DELIVERABLES AND PRICING.(a)  Vendor shall provide the Services and Deliverables and charge the fees for same as more specifically stated in Schedule 1 hereto and incorporated herein all in accordance with this SOW and the terms and conditions stated and referenced in the Master Agreement.  (b)  Vendor shall invoice AGI Corporation in accordance with the terms of the Master Agreement and the pricing schedule set forth in Schedule 1 hereto.  Vendor shall include on all invoices a reference to the Master Agreement the SOW number stated above a description of the Services to which the invoice relates and the correct fee.    2.PERSONNEL Vendor hereby appoints Bilesh Prasad as the Vendor Project Manager with respect to this SOW.  All communications with the Vendor Project Manager shall be directed to(b)  AGI Corporation hereby appoints AGI Corporation PROJECT MANAGER NAME as the AGI Corporation Project Manager with respect to this SOW.  All communications with the AGI Corporation Project Manager shall be directed to3.SCHEDULE.  The Services and the Deliverables shall be provided by Vendor to AGI Corporation in accordance with the Performance and Delivery Schedule set forth as Schedule 3 hereto. 4.TERM TERMINATION.  This SOW shall commence upon the SOW Effective Date and continue until January 28 2021 or if no expiration date is stated upon completion of the Services unless terminated in accordance with the terms hereof or the terms of the Master Agreement.  5.SERVICE LOCATIONS.  The Services shall be provided from the following Service LocationsMetropolitan Life Insurance Company Bridgewater NJLatentView Analytics Corporation Pvt. Ltd5th Floor Neville Tower Block A3Ramanujan IT City SEZRajiv Gandhi Salai (OMR) Taramani Chennai600 113 INDIA6.SERVICE LEVELS.  The Services shall be provided in accordance with the Service Levels set forth in Schedule4 hereto. 7.CHANGES TO SERVICES.(a)  The Parties may request and implement changes to the Services andor Deliverables.(b)  Vendor shall prepare a Change Order in the form substantially similar to Schedule 2 hereto for any proposed changes to the Services andor Deliverables and shall send such Change Order to AGI Corporation for written approval prior to Vendor implementing any such changes.  (c)  Except as provided in this Section no order statement or conduct of AGI Corporation shall be considered to be a change in the Services andor Deliverables or entitle Vendor to any type of equitable adjustment to the fees.7.FORCE MAJEURE.Force Majeure Eventmeans the occurrence of(a) an act of war (whether declared or not) hostilities invasion act of foreign enemies terrorism or civil disorder.(b) ionising radiations or contAGInation by radioactivity from any nuclear fuel or from any nuclear waste from the combustion of nuclear fuel radioactive toxic explosive or other hazardous properties of any explosive nuclear assembly or nuclear component thereof.(c) pressure waves from devices travelling at supersonic speeds or damage caused by any aircraft or similar device.(d) a strike or strikes or other industrial action or blockade or embargo or any other form of civil disturbance (whether lawful or not) in each case affecting on a general basis the industry related to the affected Services and which is not attributable to any unreasonable action or inaction on the part of the Company or any of its Subcontractors or suppliers and the settlement of which is beyond the reasonable control of all such persons(d) specific incidents of exceptional adverse weather conditions in excess of those required to be designed for in this Agreement which are materially worse than those encountered in the relevant places at the relevant time of year during the twenty (20) years prior to the Effective Date(e) tempest earthquake or any other natural disaster of overwhelming proportions pollution of water sources resulting from any plane crashing into(f) discontinuation of electricity supply not covered by the agreement concluded with the utility company or(g) other unforeseeable circumstances beyond the control of the Parties against which it would have been unreasonable for the affected party to take precautions and which the affected party cannot avoid even by using its best effortswhich in each case directly causes either party to be unable to comply with all or a material part of its obligations under this Agreement.(1) Neither Party shall be in breach of its obligations under this Agreement (other than payment obligations) or incur any liability to the other Party for any losses or damages of any nature whatsoever incurred or suffered by that other (otherwise than under any express indemnity in this Agreement) if and to the extent that it is prevented from carrying out those obligations by or such losses or damages are caused by a Force Majeure Event except to the extent that the relevant breach of its obligations would have occurred or the relevant losses or damages would have arisen even if the Force Majeure Event had not occurred (in which case this Clause 20 shall not apply to that extent).(2) As soon as reasonably practicable following the date of commencement of a Force Majeure Event and within a reasonable time following the date of termination of a Force Majeure Event any Party invoking it shall submit to the other Party reasonable proof of the nature of the Force Majeure Event and of its effect upon the performance of the Partys obligations under this Agreement.(3) The Company shall and shall procure that its Subcontractors shall at all times take all reasonable steps within their respective powers and consistent with Good Operating Practices (but without incurring unreasonable additional costs) to(a) prevent Force Majeure Events affecting the performance of the Companys obligations under this Agreement.(b) mitigate the effect of any Force Majeure Event and(c) comply with its obligations under this Agreement.The Parties shall consult together in relation to the above matters following the occurrence of a Force Majeure Event.(4) and other communications hereunder shall be given in writing to the parties within 30 business days (5) Should paragraph (1) apply as a result of a single Force Majeure Event for a continuous period of more than 60 days then the parties shall endeavor to agree any modifications to this Agreement (including without limitation determination of new tariffs (if appropriate) in accordance with the provisions of Clause 7(4)(e)) which may be equitable having regard to the nature of the Force Majeure Event and which is consistent with the Statutory Requirements.Schedule 1 Services Deliverables and Pricing Description of Services  Vendor will provide the following Services to AGI CorporationServices ParametersCreate a selfserve BI Implementation for the 18 identified reports around disability claims  closure. The service scope includes the following parametersCreating a data mart for all required data elements Identifyingvalidating the trusted source system for all the data elements Creating a selfservice application for end users with varying skill levelsBuild visualizations and provide insights into the disability data for business users ServicesCreating a Data Mart For All Required Data Elements List of data elements that need to be available in the Self Service ApplicationBI Tool has to be preparedConduct informal stakeholder discussions to understand and document the end user requirementsAssess the quality coverage and consistency of each data attribute and its value on each KPI within the dashboardIdentifyingvalidating the trusted source system for all the data elements Identify the trusted data source from various data sources.Streamline data extraction through the implementation of BI applicationCreating a selfservice application for end users with varying skill levelsBI application has to cater to two different kinds of usersStandard reports accessible to basic usersAdhoc nonstandard reports that can be generated by power users per their requirement.Build visualizations and provide insights into the disability data for business users The reports under the current scope which are to redesigned automated and converted to selfserve mode areContd..Deliverables  As part of the Services Vendor will provide the following Deliverables to AGI Corporation in accordance with the Performance and Delivery Schedule. Requirements document focusing on data elements needed for standard reports.Logical  physical data model for Data Mart.Data mapping document for ETL from Data sources into the Data Mart Design document including system diagramsProject planProject charter (Completed in the Initiation Phase)Scope statement Work breakdown structure (WBS) Cost estimates schedule responsibilities for each deliverable Milestones with target dates Risk Issues register Management plans (scope schedule cost quality communications risk and procurement)Standard reports and dashboards developed using tools like TableauTraining Instruction and Related Support. Knowledge Transition and Training will be provided on a need basis.Acceptance  Acceptance of Deliverables shall be in accordance with the terms of the Master Agreement. AGI Corporation Obligations and Resources  AGI Corporation shall be responsible for providing the following services hardware software or other materialsApplicable data files file layouts and associated data dictionary for each report designated for this project includingOrganization Chart of Data Source OwnersInitial List of Data SourcesAny previous data quality assessmentsAccess to list of systems and existing ReportsCurrent business calculations for existing KPIs and reports including series of data and supporting business rules and programs that create data points for reportsBusiness Information GlossaryAccess to the infrastructure to build selfserve module which includes Access to data and sources for both LatentView onsite and offshore teamsRequired client and server infrastructure and associated licenses for the BI layer (For eg. Access to Tableau Server and developer licenses for the team)Access and availability of AGI Corporation personnel who can answer questions related to data sources file layouts and calculationsPricing  Vendor shall charge AGI Corporation and AGI Corporation shall pay Vendor for Services rendered and Deliverables associated therewith as followsThe local currency applicable to this SOW is USD. The Services described in this SOW shall be provided on a fixed fee basis is 200000 will be invoiced at as per the schedule below. The incidental travel boarding and lodging expenses will be billed on actuals and shall not exceed 10 of the invoice amount. Any extensions to the current scope and project schedule based on the findings of the Analysis  Planning Phase will be invoiced on a TM basis according to the rates mentioned in the MSA once the parties mutually agree on the increased scope and the supplier billable hours required to complete it. These modifications as well as modifications made at any time during the SOW will follow the change order process in Schedule 2. Travel  Expenses Except as otherwise set forth in this SOW or in the Master Agreement no expenses shall be payable by AGI Corporation with respect to the Services and Deliverables to be provided pursuant to this SOW. Any reasonable and customary travel and outofpocket expenses payable hereunder will be charged at cost (after Vendor deducts all discounts and rebates it receives) in no event exceed 15 of the total invoice amount for outofpocket expenses but not necessarily limited to travel and lodging expenses communications charges and computer time and supplies and taxes as applicable.  AGI Corporation shall not be responsible for any additional charges unless stated otherwise in an amendment to this SOW or a Change Order appropriately signed by the Parties.Schedule 3  Performance and Delivery ScheduleThe following table list down the servicestasks that shall be delivered with the timeline. Delivery plan shall be baselined after the final scope is agreed upon at the end of the Analysis and Planning Phase.The following are the indicative (not exhaustive) deliverables for the servicesSchedule 4  Service LevelsVendor shall complete the project and deliverables and deliver to AGI Corporation on or before the agreed timelines at the end of the Analysis  Planning Phase. LatentView will meet the AGI Corporation stakeholders at regular intervals and provide status updates to enhance clarity and transparency.The above mentioned services will be provided by LatentView to AGI Corporation based on mutually agreed KPIs listed in below table In case of any unforeseen data outage  platform related issues the delivery timelines will be adjusted accordingly. Other planned deliverables will also be reprioritized and communicated to the respective stakeholders."
doc = nlp(test_text)
print("Entities in '%s'" % test_text)
for ent in doc.ents:
        print(ent.label_, ent.text)

Entities in 'SCHEDULE D FORM OF STATEMENT OF WORKStatement of Work Number PREAMBLEThis Statement of Work (SOW) effective as of 01st June 2019 (the SOW Effective Date) is entered into by and between AGI Corporation and Latentview Analytics Corporation  (Vendor) pursuant to the terms and conditions set forth in the Master Agreement.WHEREAS AGI Corporation issued and Vendor responded to a request for proposal (RFP) entitled) Disability RFPRFI Inquiry SelfService to which Vendor affirmatively responded and stated that it is fully capable and willing to provide Business intelligence implementation services services as more fully described in this SOW (the Services).WHEREAS AGI Corporation would like Vendor to provide and Vendor would like to provide the services as more fully described in this SOW (the Services). NOW THEREFORE in consideration of the foregoing and the mutual promises covenants and agreements set forth below and in the Master Agreement and for other good and valuable conside

In [48]:
nlp.pipe_labels

{'ner': ['Liability', 'Notification_type', 'Termination_period']}

In [49]:
model_path = 'contract_ner_model\\'

In [50]:
new_model_name = 'en_contarct_ner'
nlp.meta["name"] = new_model_name  # rename model

model_path =Path(model_path)
if not model_path.exists():
    model_path.mkdir()

nlp.to_disk(model_path)
print("Saved model ",new_model_name)

Saved model  en_contarct_ner
